In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from itertools import combinations
from math import comb
from time import time
from datetime import date
import warnings
from joblib import Parallel, delayed
import json

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, plot_confusion_matrix, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity

warnings.filterwarnings('ignore')
# pd.set_option('mode.chained_assignment', None)

# Retrofitting Pre-Req Class Definitions

## Utils

In [3]:
class Utils:
    """
    This contains all the utility functions needed by any part of retrofitting
    """
    _today = date.today()
    today_date = _today.strftime("%b_%d_%Y")
    LABELS = ['I','M','U']
    
    @classmethod
    def normalize(cls, embed_dict):
        for key, val in embed_dict.items():
            temp = np.array([float(val1) for val1 in val])
            temp2 = temp**2
            embed_dict[key] = temp / np.sqrt((temp2.sum() + 1e-6))
        return embed_dict
    
    @classmethod
    def fetch_embeddings(cls, df):
        embed_dict = {}
        for _, row in df.iterrows():
            embed_dict[row.node] = row.value
        return normalize(embed_dict)
    
    @classmethod
    def fill_coverage(cls, embed_dict, embed_name):
        wordsim_df = pd.read_csv('../data/wordsim353_with_r3.csv')
        wiki_cs_df = pd.read_csv('../data/wikidata-cs_categorized.csv')
        concept_net_df = pd.read_csv('../data/kgtk_conceptnet_evaluation.csv')
        
        compulsory_coverage_set = set(
                        wordsim_df['word1_kg_id'].to_list() 
                        + wordsim_df['word2_kg_id'].to_list()
                        + wiki_cs_df['word1_kg_id'].to_list() 
                        + wiki_cs_df['word2_kg_id'].to_list()
                        + concept_net_df['word1_kg_id'].to_list()
                        + concept_net_df['word2_kg_id'].to_list())
        
        embed_size = len(embed_dict[next(iter(embed_dict))])
        count = 0
        for word in compulsory_coverage_set:
            if word not in embed_dict:
                embed_dict[word] = np.zeros((embed_size))
                count += 1
        print(f"Added {count} corrections to {embed_name}")
        return embed_dict
    
    @classmethod
    def check_coverage(cls, embed_dict):
        wordsim_df = pd.read_csv('../data/wordsim353_with_r3.csv')
        
        compulsory_coverage_set = set(wordsim_df['word1_kg_id'].to_list() + wordsim_df['word2_kg_id'].to_list())
        embed_size = len(embed_dict[next(iter(embed_dict))])
        count = 0
        for word in compulsory_coverage_set:
            if word not in embed_dict:
                count += 1
        return (len(wordSim353AnnotDF_New) - count)
    
    @classmethod
    def determine_distances(cls, embed_dict, new_embed_dict):
        dist = []
        for word in embed_dict.keys():
            dist.append(euclidean_distances([embed_dict[word]], [new_embed_dict[word]])[0][0])
        return dist
    
    @classmethod
    def serialize_embedding_dict(cls, embed_dict):
        for key2 in embed_dict.keys():
            embed_dict[key2] = embed_dict[key2].tolist() if type(embed_dict[key2]) != list else embed_dict[key2]
        return embed_dict
    
    @classmethod
    def deserialize_embedding_dict(cls, embed_dict):
        for key2 in embed_dict.keys():
            embed_dict[key2] = np.array(embed_dict[key2])
        return embed_dict
    
    @classmethod
    def label_samples(cls, score):
        return 'I' if score <= 1.75 else 'U' if score >= 3.5 else 'M'
    
    @classmethod
    def determine_cos_sim(cls, emb1, emb2):
        return cosine_similarity(
                np.array(emb1).reshape(1,-1), 
                np.array(emb2).reshape(1,-1)
            )[0][0]
    
    @classmethod
    def plot_confusion_matrix(cls, conf_matrix, title):
        plt.close()
        sns.heatmap(conf_matrix, xticklabels=Utils.LABELS, yticklabels=Utils.LABELS, annot=True)
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title(title+' Confusion Matrix')

In [4]:
# import sys
# def sizeof_fmt(num, suffix='B'):
#     ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
#     for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
#         if abs(num) < 1024.0:
#             return "%3.1f %s%s" % (num, unit, suffix)
#         num /= 1024.0
#     return "%.1f %s%s" % (num, 'Yi', suffix)

#     for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
#                              key= lambda x: -x[1])[:10]:
#         print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

## InputEmbeddings

In [5]:
class InputEmbeddings:
    """
    Instance variables:
        - embed_dict_master - holds all qnode to embedding mappings as a dictionary
        - embedding_list - list of all keys of the above dictionary
    """
    def __init__(self, has_embeddings_include: bool = True):
        self.embed_dict_master = {}
        self.emb_list = ['text_7_props', 'text_2_props', 'complex', 'transe', 'abstract', 'abstract_first_sent']
        if has_embeddings_include: # TODO
            self.emb_list += ['has_h', 'has_a', 'has_s']
        self.embedding_lengths = {}
        
        for emb_key in tqdm(self.emb_list, desc='Input Embeddings', leave=False):
            self.embed_dict_master[emb_key] = self.fetch_embedding(emb_key)
            
        self.fetch_embedding_stats()
        print("Fetched all input embeddings")

    def fetch_embedding(self, emb_key):
        return Utils.fill_coverage(
                Utils.deserialize_embedding_dict(
                    json.load(open('../data/Master_P279_dataset/embeddings/'+emb_key+'_orig_embedding_dict.json'))
                ), emb_key
            )

    def fetch_embedding_stats(self):
        for emb_name in self.embed_dict_master.keys():
            self.embedding_lengths[emb_name] = len(next(iter(self.embed_dict_master[emb_name].values())))
            print(f"Embedding: {emb_name}, Size: {len(self.embed_dict_master[emb_name].keys())}, Length: {self.embedding_lengths[emb_name]}")

## NeighborDatasets

In [16]:
class NeighborDatasets:
    """
    Instance variables:
        - neighbors_dict_master - holds all qnode to neighbor qnode mappings as a dictionary
        - basis_list - list of all keys of the above dictionary
    """
    
    def __init__(self, class_datasets_fetch: bool = True, probase_datasets_fetch: bool = False):
        self.neighbors_dict_master = {}
        
        pbar = tqdm(desc='Neighbor Datasets', leave=False, total = 
                    6 
                    + (3 if class_datasets_fetch else 0) 
                    + (2 if probase_datasets_fetch else 0) 
                   )
        
        bert_P279_child_par_df = pd.read_csv('../data/Master_P279_dataset/P279ChildPar_filtered.csv')
        bert_P279_child_par_df_cross_enc = pd.read_csv('../data/Master_P279_dataset/P279ChildPar_filtered_cross_enc.csv')
        bert_P279_siblings_df = pd.read_csv('../data/Master_P279_dataset/P279Siblings_transP279_filtered_min_cols_with_desc_dups_removed.csv')
        bert_P279_siblings_df_cross_enc = pd.read_csv('../data/Master_P279_dataset/P279Siblings_transP279_filtered_min_cols_with_desc_dups_removed_cross_enc.csv')
        
        self.neighbors_dict_master['bert_child_par'] = self.fetch_neighbours(bert_P279_child_par_df)
        pbar.update(1)
        self.neighbors_dict_master['bert_siblings'] = self.fetch_neighbours(bert_P279_siblings_df)
        pbar.update(1)
        self.neighbors_dict_master['bert_all'] = self.fetch_neighbours(pd.concat([
                bert_P279_child_par_df, bert_P279_siblings_df
            ]))
        pbar.update(1)
        
        self.neighbors_dict_master['cross_enc_child_par'] = self.fetch_neighbours(bert_P279_child_par_df)
        pbar.update(1)
        self.neighbors_dict_master['cross_enc_siblings'] = self.fetch_neighbours(bert_P279_siblings_df)
        pbar.update(1)
        self.neighbors_dict_master['cross_enc_all'] = self.fetch_neighbours(pd.concat([
                bert_P279_child_par_df, bert_P279_siblings_df
            ]))
        pbar.update(1)
            
        if class_datasets_fetch:
            class_P279_child_par_df = pd.read_csv('../data/Master_P279_dataset/P279ChildPar_transP279_filtered_w_classSim.csv')
            class_P279_child_par_df['similarity_value'] = class_P279_child_par_df['classSim']
            
            class_P279_siblings_df = pd.read_csv('../data/Master_P279_dataset/P279Siblings_transP279_filtered_w_classSim.csv')
            class_P279_siblings_df['similarity_value'] = class_P279_siblings_df['classSim']
            
            self.neighbors_dict_master['class_child_par'] = self.fetch_neighbours(class_P279_child_par_df)
            pbar.update(1)
            self.neighbors_dict_master['class_siblings'] = self.fetch_neighbours(class_P279_siblings_df)
            pbar.update(1)
            self.neighbors_dict_master['class_all'] = self.fetch_neighbours(pd.concat([
                    class_P279_child_par_df, class_P279_siblings_df
                ]))
            pbar.update(1)

        if probase_datasets_fetch:
            probase_df = self.fetch_probase('../data/probase/probase_WQnodes_subset_and_sim.csv')
            
            self.neighbors_dict_master['probase'] = self.fetch_neighbours(probase_df)
            pbar.update(1)
            self.neighbors_dict_master['probase+bert_all'] = self.fetch_neighbours(pd.concat([
                    bert_P279_child_par_df, bert_P279_siblings_df, probase_df
                ]))
            pbar.update(1)
        
        self.basis_list = list(self.neighbors_dict_master.keys())
        
        pbar.close()

        print(f"Fetched neighbour datasets: {self.basis_list}")
    
    def process_probase(self, probase_file_path):
        probase_df = pd.read_csv(probase_file_path)
        probase_df = probase_df.rename(columns={'n1_final_qnode': 'node1', 'n2_final_qnode': 'node2', 'sim': 'similarity_value'})
        probase_df['similarity_value'] = 0.5 + 0.5 * probase_df['similarity_value']
        
        return probase_df
        
    def fetch_neighbours(self, df):
        neighbours_dict = {}
        for _, row in df.iterrows():
            if row.node1 not in neighbours_dict:
                neighbours_dict[row.node1] = []
            neighbours_dict[row.node1].append((row.node2, row.similarity_value))

            if row.node2 not in neighbours_dict:
                neighbours_dict[row.node2] = []
            neighbours_dict[row.node2].append((row.node1, row.similarity_value))
#         print(max([len(neigh) for neigh in neighbours_dict.values()]))
        
        return neighbours_dict

## EvaluationDatasets

In [7]:
class EvaluationDatasets:
    def __init__(self):
        self.wordsim_df = pd.read_csv('../data/wordsim353_with_r3.csv')
        self.fetch_distribution_stats("Wordsim-353", self.wordsim_df)
        
        self.wiki_cs_df = pd.read_csv('../data/wikidata-cs_categorized.csv')
        self.fetch_distribution_stats("Wikidata CS", self.wiki_cs_df)
        
        self.concept_net_df = pd.read_csv('../data/kgtk_conceptnet_evaluation.csv')
        self.fetch_distribution_stats("Concept Net", self.concept_net_df)
        
        self.get_coverage_nodes()
        
    def fetch_distribution_stats(self, name, dataset):
        print(f"Dataset: {name}")
        print(dataset.category.value_counts())
    
    def get_coverage_nodes(self):
        self.coverage = set(
                        self.wordsim_df['word1_kg_id'].to_list() 
                        + self.wordsim_df['word2_kg_id'].to_list()
                        + self.wiki_cs_df['word1_kg_id'].to_list() 
                        + self.wiki_cs_df['word2_kg_id'].to_list()
                        + self.concept_net_df['word1_kg_id'].to_list()
                        + self.concept_net_df['word2_kg_id'].to_list())

## ResultMetrics

In [8]:
class ResultMetrics:
    
    @classmethod
    def compute_classification_results(cls,
            embed_dict, 
            eval_dataset,
            get_output_values: bool = False,
            old_accuracy = None
            ):
        """
        Inputs:
            - embed_dict - dictionary of qnodes with node embeddings as its values
            - eval_dataset - evaluation dataset as pandas dataframe that must have the 
                following columns for this function to work correctly:
                * word1_kg_id - Qnode of node1 in the evaluation pair
                * word2_kg_id - Qnode of node2 in the evaluation pair
                * category - Category of the evaluation pair. One of the labels: I/U/M
        Outputs:
            - response_dict - Returns a dictionary with the following keys:
                * covered_pairs - Indicates the number of pairs of the evaluation dataset that the 
                    embedding dictionary can cover
                
        """
        response_dict = {}
        
        eval_dataset = eval_dataset.copy()

        missing_words_set = set(
            eval_dataset[eval_dataset.word1_kg_id.apply(lambda p: p not in embed_dict)].word1_kg_id.to_list() 
            + eval_dataset[eval_dataset.word2_kg_id.apply(lambda p: p not in embed_dict)].word2_kg_id.to_list()
        )
        
        response_dict['covered_pairs'] = len(eval_dataset)

        eval_dataset['embedding_cos_sim'] = eval_dataset.apply(lambda p: Utils.determine_cos_sim(embed_dict[p['word1_kg_id']], embed_dict[p['word2_kg_id']]) 
                                                   if p['word1_kg_id'] in embed_dict and p['word2_kg_id'] in embed_dict 
                                                   else None, axis=1)
        
        eval_dataset['embedding_cos_sim'].fillna(eval_dataset['embedding_cos_sim'].mean(skipna=True), inplace=True)
        
        # Scale abs value of cosine similarities to 1,4 strictly
        eval_dataset['embedding_cos_sim'] = eval_dataset['embedding_cos_sim'].apply(lambda p: 4 - 3 * abs(p))

        response_dict['accuracy'] = 100 * accuracy_score(
                eval_dataset['category'],
                eval_dataset['embedding_cos_sim'].apply(Utils.label_samples)
            )
    
        response_dict['classification_report'] = classification_report(
                eval_dataset['category'], 
                eval_dataset['embedding_cos_sim'].apply(Utils.label_samples), 
                output_dict=True
            )

        response_dict['conf_matrix'] = confusion_matrix(
                eval_dataset['category'], 
                eval_dataset['embedding_cos_sim'].apply(Utils.label_samples), 
                labels=Utils.LABELS
            )
        
        if old_accuracy is not None:
            response_dict['increase_acc'] = response_dict['accuracy'] - old_accuracy
        else:
            response_dict['increase_acc'] = None
        
        if get_output_values:
            response_dict['preds'] = eval_dataset['embedding_cos_sim'].apply(Utils.label_samples)

        return response_dict, \
                (response_dict['covered_pairs'],  \
                 response_dict['accuracy'], \
                 response_dict['increase_acc'], \
                 
                 response_dict['classification_report']['I']['precision'],  \
                 response_dict['classification_report']['I']['recall'],  \
                 response_dict['classification_report']['I']['f1-score'],   \
                 
                 response_dict['classification_report']['M']['precision'],  \
                 response_dict['classification_report']['M']['recall'],  \
                 response_dict['classification_report']['M']['f1-score'], \
                 
                 response_dict['classification_report']['U']['precision'],  \
                 response_dict['classification_report']['U']['recall'],  \
                 response_dict['classification_report']['U']['f1-score'])
    
    @classmethod
    def fetch_best_result_for_emb(cls, results_df, emb_col, target_col, iter_col, highest: bool = True):
        opt_value = {}
        for _, row in results_df.iterrows():
            if row[emb_col] not in opt_value:
                opt_value[row[emb_col]] = {'opt_metric': float('-inf') if highest else float('inf'),
                                     'opt_row': [], 'old_row': []}
            if row[iter_col] == 0:
                opt_value[row[emb_col]]['old_row'] = row
            else:
                if (highest and row[target_col] > opt_value[row[emb_col]]['opt_metric']) \
                        or (not(highest) and row[target_col] < opt_value[row[emb_col]]['opt_metric']):
                    opt_value[row[emb_col]]['opt_metric'] = row[target_col]
                    opt_value[row[emb_col]]['opt_row'] = row
        best_results = []
        for emb_key in opt_value:
            best_results.append(opt_value[emb_key]['old_row'])
            best_results.append(opt_value[emb_key]['opt_row'])
        return pd.DataFrame(best_results, columns = results_df.columns)

## RetrofittingProcedures

In [9]:
class RetrofittingProcedures:
    
    np_label_samples = np.vectorize(Utils.label_samples)
    
    @classmethod
    def retrofit(cls,embed_dict, neighbors_dict, weight_case, weight_assignment=False):
        new_embed_dict = {}
        for word in embed_dict.keys():
            if word in neighbors_dict:
                neighbs = neighbors_dict[word]
                neighbs = list(filter(lambda p: p[0] in embed_dict, neighbs))
                if len(neighbs) == 0:
                    new_embed_dict[word] = embed_dict[word]
                    continue
                if weight_assignment:
                    sum_of_sims = sum([neighb[1] for neighb in neighbs])
                    sum_of_embs = sum([embed_dict[neighb[0]] * float(neighb[1]) for neighb in neighbs])
                else:
                    sum_of_sims = sum([1 for neighb in neighbs])
                    sum_of_embs = sum([embed_dict[neighb[0]] for neighb in neighbs])

                if weight_case == 1:
                    new_embed_dict[word] = (embed_dict[word] * (len(neighbs)) + sum_of_embs) / ((len(neighbs)) + sum_of_sims)
                elif weight_case == 2:
                    new_embed_dict[word] = (embed_dict[word] * (len(neighbs))**2 + sum_of_embs) / ((len(neighbs))**2 + sum_of_sims)
                elif weight_case == 0.5:
                    new_embed_dict[word] = (embed_dict[word] * (len(neighbs))**0.5 + sum_of_embs) / ((len(neighbs))**0.5 + sum_of_sims)
                else:
                    raise
            else:
                new_embed_dict[word] = embed_dict[word]
        return new_embed_dict
    
    @classmethod
    def execute_all_unsupervised_scenarios(cls,
                emb_list, basis_list, 
                embed_dict_master, neigh_dict_master,
                eval_dataset,
                scenario_name: str,
                num_of_iterations: int = 3, 
                weightedness_list: list = [True],
                weight_cases_list: list = [1,2],
                get_output_values: bool = False
            ):
        
        new_embed_dict_master = {}
        responses_dict_master = {}
        results = []
        
        for basis in tqdm(basis_list, desc='Basis', leave=False):
            for emb in tqdm(emb_list, desc='Embedding', leave=False):
                for weightedness in weightedness_list:
                    for weight_case in tqdm(weight_cases_list, desc='Weight Case', leave=False):
                        # Base Reference Initializations and Calculations
                        embed_dict = embed_dict_master[emb]
                        responses_dict, result_values = ResultMetrics.compute_classification_results(
                            embed_dict, eval_dataset, get_output_values=get_output_values, old_accuracy=None)
                        results.append([emb, basis, weight_case, weightedness, 0, 'base', *result_values])
                        old_accuracy = responses_dict['accuracy']
                        
                        for iter_num in tqdm(range(1,num_of_iterations+1), desc='Iteration', leave=False):
                            start_time = time()
                            
                            case_name = emb + '_' + basis + '_' + str(weight_case) + ('_weighted' if weightedness else '_unweighted')
                            
                            new_embed_dict = cls.retrofit(embed_dict, neigh_dict_master[basis], weight_case, weightedness)
                            
                            responses_dict, result_values = ResultMetrics.compute_classification_results(
                                new_embed_dict, eval_dataset, get_output_values=get_output_values, old_accuracy=old_accuracy)
                            
                            results.append([emb, basis, weight_case, weightedness, iter_num, case_name, \
                                                *result_values, \
                                                time() - start_time
                                            ])
                            
                            new_embed_dict_master[case_name] = embed_dict = new_embed_dict
                            responses_dict_master[case_name] = responses_dict

        #                     if iter_num == num_of_iterations and highestOne:
        #                         case_name = gR[0] + '_' + gR[1] + '_' + str(gR[2]) + '_weighted'
        #                         new_embed_dict_master[case_name] = serializeEmbeddingDict(new_embed_dict_master[case_name])
        #                         highestOne = False
        #                         json.dump(new_embed_dict_master[case_name],open('../data/Master_P279_dataset/embeddings/new_embedding_dict_'+case_name+'.json','w'))
        #                         new_embed_dict_master[case_name] = deserializeEmbeddingDict(new_embed_dict_master[case_name])
        resultsDF = pd.DataFrame(results, columns=['Embedding', 'Basis', 'Weight Case', 'Weightedness', 
                                                   'Iteration Num', 'Case Name', \
                                                   'No. of Pairs Covered', 'Accuracy', 'Increase in Accuracy', \
                                                   'I Precision', 'I Recall', 'I F1-Score', \
                                                   'M Precision', 'M Recall', 'M F1-Score', \
                                                   'U Precision', 'U Recall', 'U F1-Score', \
                                                   'Time to Retrofit'])
        resultsDF.to_csv('../data/retrofitting/retro_unsup_results.' + scenario_name + '.'+ Utils.today_date +'.csv', index=False)
        best_results_df = ResultMetrics.fetch_best_result_for_emb(resultsDF, 'Embedding', 'Accuracy', 'Iteration Num', highest=True)
        best_results_df.to_csv('../data/retrofitting/retro_unsup_results.' + scenario_name + '.'+ Utils.today_date +'.best.csv', index=False)
        
        cls.save_needed_embeddings(new_embed_dict_master)
        
        return new_embed_dict_master, responses_dict_master
    
    @classmethod
    def save_needed_embeddings(cls, new_embed_dict_master):
        for case_name in new_embed_dict_master:
            temp = {key: new_embed_dict_master[case_name][key] for key in new_embed_dict_master[case_name] if key in evalD.coverage}
            new_embed_dict_master[case_name] = temp

## SVMProcedures

In [10]:
class SVMProcedures:
    @classmethod
    def execute_supervised_scenario(cls,
                eval_dataset, case, embed_dict_master, new_embed_dict_master, 
                num_of_splits = 10, 
                comb_mode: bool = False, SVC_or_SVR: str = 'SVC'
            ):
        
        X = []        
        
        ################
        # 2 Approaches based on argument: `comb_mode`
        ################
        
        if comb_mode: ########## COMBINATION MODE CODE ####################
            case_name = " & ".join(case['emb']) + '_' + case['basis'] + '_' + str(case['weight_case']) + ('_weighted' if case['weightedness'] else '_unweighted')
            
            for _, row in eval_dataset.iterrows():
                if case['svm_input'] == 'emb':
                    tempX = []
                    for individual_emb in case['emb']:
                        ind_case_name = individual_emb + '_' + case['basis'] + '_' + str(case['weight_case']) + ('_weighted' if case['weightedness'] else '_unweighted')
                        if case['iter_num'] != 0 and ind_case_name not in new_embed_dict_master:
                            return case_name, case, None
                        if case['iter_num'] == 0:
                            tempX += embed_dict_master[individual_emb][row['word1_kg_id']].tolist() + embedDictMaster[individual_emb][row['word2_kg_id']].tolist()
                        else:
                            tempX += new_embed_dict_master[ind_case_name][row['word1_kg_id']].tolist() + newEmbedDictMaster[ind_case_name][row['word2_kg_id']].tolist()
                    X.append(tempX)
                else:
                    tempX = []
                    for individual_emb in case['emb']:
                        ind_case_name = individual_emb + '_' + case['basis'] + '_' + str(case['weight_case']) + ('_weighted' if case['weightedness'] else '_unweighted')
                        if case['iter_num'] != 0 and ind_case_name not in new_embed_dict_master:
                            return case_name, case, None
                        if case['iter_num'] == 0:
                            tempX.append(abs(Utils.determine_cos_sim(
                                embed_dict_master[individual_emb][row['word1_kg_id']], 
                                embed_dict_master[individual_emb][row['word2_kg_id']]
                            )))
                        else:
                            tempX.append(abs(Utils.determine_cos_sim(
                                new_embed_dict_master[ind_case_name][row['word1_kg_id']],
                                new_embed_dict_master[ind_case_name][row['word2_kg_id']]
                            )))
                    X.append(tempX)

        else: ########## NON-COMBINATION MODE CODE ####################
            case_name = case['emb'] + '_' + case['basis'] + '_' + str(case['weight_case']) + ('_weighted' if case['weightedness'] else '_unweighted')
            if case['iter_num'] != 0 and case_name not in new_embed_dict_master:
                return case_name, case, None
            for _, row in eval_dataset.iterrows():
                if case['svm_input'] == 'emb':
                    if case['iter_num'] == 0:
                        X.append(embed_dict_master[case['emb']][row['word1_kg_id']].tolist() + embed_dict_master[case['emb']][row['word2_kg_id']].tolist())
                    else:
                        X.append(new_embed_dict_master[case_name][row['word1_kg_id']].tolist() + new_embed_dict_master[case_name][row['word2_kg_id']].tolist())
                else:
                    if case['iter_num'] == 0:
                        X.append(abs(Utils.determine_cos_sim(
                                embed_dict_master[case['emb']][row['word1_kg_id']], 
                                embed_dict_master[case['emb']][row['word2_kg_id']]
                            )))
                    else:
                        X.append(abs(Utils.determine_cos_sim(
                                new_embed_dict_master[case_name][row['word1_kg_id']],
                                new_embed_dict_master[case_name][row['word2_kg_id']]
                            )))
                    
        X = pd.DataFrame(X)
        
        ################
        # 2 Approaches based on argument: `SVC_or_SVR`
        ################
        
        # Target split depending on SVC or SVM
        if SVC_or_SVR == 'SVC':
            Y = eval_dataset['category']
        elif SVC_or_SVR == 'SVR':
            if 'Avg' not in eval_dataset.columns:
                raise ValueError("Avg column not present in the provided eval_dataset")
            Y = (eval_dataset['Avg'] - 1) / 3
        else:
            raise ValueError("Invalid SVC_or_SVR provided")
        
        if SVC_or_SVR == 'SVC':
            skf = StratifiedKFold(n_splits=num_of_splits, random_state=19, shuffle=True)
            X_train_splits, X_test_splits, Y_train_splits, Y_test_splits = [], [], [], []
            for train_index, test_index in skf.split(X, Y):
                X_train_splits.append(X.iloc[train_index])
                X_test_splits.append(X.iloc[test_index])
                Y_train_splits.append(Y.iloc[train_index])
                Y_test_splits.append(Y.iloc[test_index])
        elif SVC_or_SVR == 'SVR':
            skf = KFold(n_splits=num_of_splits, random_state=19, shuffle=True)
            X_train_splits, X_test_splits, Y_train_splits, Y_test_splits = [], [], [], []
            for train_index, test_index in skf.split(X, Y):
                X_train_splits.append(X.iloc[train_index])
                X_test_splits.append(X.iloc[test_index])
                Y_train_splits.append(Y.iloc[train_index])
                Y_test_splits.append(Y.iloc[test_index])

        preds = []
        
        # Classifier/Regressor training depending on SVC or SVM
        if SVC_or_SVR == 'SVC':
            for X_train1, Y_train1, X_test1, Y_test1 in zip(X_train_splits, Y_train_splits, X_test_splits, Y_test_splits):
                clf = make_pipeline(StandardScaler(), SVC(gamma='auto', random_state=100, max_iter=100))
                clf.fit(X_train1, Y_train1)
                preds.append(clf.predict(X_test1))
                
            acc = 0
            for pred, Y_test1 in zip(preds, Y_test_splits):
                acc += accuracy_score(pred, Y_test1)

            return case_name, *list(case.values()), acc/num_of_splits
        
        elif SVC_or_SVR == 'SVR':
            for X_train1, Y_train1, X_test1, Y_test1 in zip(X_train_splits, Y_train_splits, X_test_splits, Y_test_splits):
                clf = make_pipeline(StandardScaler(), SVR(gamma='auto', max_iter=100))
                clf.fit(X_train1, Y_train1)
                preds.append(clf.predict(X_test1))
            
            acc = 0
            ktCorr = 0
            for pred, Y_test1 in zip(preds, Y_test_splits):
                acc += mean_squared_error(pred * 3 + 1, Y_test1 * 3 + 1)
                ktCorr += stats.kendalltau(Y_test1 * 3 + 1, pred * 3 + 1).correlation
                
            return case_name, *list(case.values()), acc/num_of_splits, ktCorr/num_of_splits
    
    @classmethod
    def execute_all_supervised_scenarios(cls,
                emb_list, basis_list, embed_dict_master, new_embed_dict_master, 
                eval_dataset, 
                scenario_name: str,
                num_of_splits = 10,
                comb_mode: bool = False, SVC_or_SVR: str = 'SVC', 
                num_of_iterations = 3,
                num_of_jobs = 1
            ):
        
        if not(comb_mode):
            svm_cases_list = []
            for basis in basis_list:
                for emb in emb_list:
                    for weightedness in [True]:
                        for iter_num in range(0,num_of_iterations+1):
                            for weight_case in [1,2]:
                                for svm_input in ['emb','score']:
                                    temp_dict = {'basis': basis, 'emb': emb, 'weightedness': weightedness, 
                                                'iter_num': iter_num, 'weight_case': weight_case, 'svm_input': svm_input}
                                    svm_cases_list.append(temp_dict) 
        else:
            svm_cases_list = []
            for basis in basis_list:
                for emb in emb_list:
                    for weightedness in [True]:
                        for iter_num in range(0,num_of_iterations+1):
                            for weight_case in [1,2]:
                                for svm_input in ['score']:
                                    for i in range(1,len(emb_list)+1):
                                        for emb_comb in combinations(emb_list, i):
                                            temp_dict = {'basis': basis, 'emb': emb_comb, 'weightedness': weightedness, 
                                                        'iter_num': iter_num, 'weight_case': weight_case, 'svm_input': svm_input}
                                            svm_cases_list.append(temp_dict) 

        results = Parallel(n_jobs=num_of_jobs)(delayed(cls.execute_supervised_scenario)(
                eval_dataset, caseDict, embed_dict_master, 
                new_embed_dict_master,num_of_splits,
                comb_mode, SVC_or_SVR
            ) for caseDict in tqdm(svm_cases_list))
        
        if SVC_or_SVR == 'SVC':
            results_df = pd.DataFrame(results, columns=['Case Name','Basis','Embedding','Weightedness', 'Iteration Num', 'Weight Case', 'Technique','Accuracy'])
            best_results_df = ResultMetrics.fetch_best_result_for_emb(results_df, 'Embedding', 'Accuracy', 'Iteration Num', highest=True)
            best_results_df.to_csv('../data/retrofitting/retro_SVM_results.' + scenario_name + '.'+ Utils.today_date +'.best.csv', index=False)
        else:
            results_df = pd.DataFrame(results, columns=['Case Name','Basis','Embedding','Weightedness', 'Iteration Num', 'Weight Case', 'Technique','MSE', 'KT Correlation'])
            best_results_df = ResultMetrics.fetch_best_result_for_emb(results_df, 'Embedding', 'MSE', 'Iteration Num', highest=False)
            best_results_df.to_csv('../data/retrofitting/retro_SVM_results.' + scenario_name + '.'+ Utils.today_date +'.best.csv', index=False)
            
        results_df.to_csv('../data/retrofitting/retro_SVM_results.' + scenario_name + '.'+ Utils.today_date +'.csv', index=False)

# The Master Controller

In [17]:
%%time
start_master_time = time()
new_embed_dict_master, responses_dict_master = {}, {}

# Load all supporting files
# inp = InputEmbeddings()
basis = NeighborDatasets()
evalD = EvaluationDatasets()

# get_size_of_variables()

# Wordsim executions
new_embed_dict_master['wordsim'], responses_dict_master['wordsim'] = RetrofittingProcedures.execute_all_unsupervised_scenarios(inp.emb_list, basis.basis_list, inp.embed_dict_master, basis.neighbors_dict_master, 
                                                          evalD.wordsim_df, "wordsim_ind")
# get_size_of_variables()
print("Analysed wordsim_ind")

SVMProcedures.execute_all_supervised_scenarios(inp.emb_list, basis.basis_list, inp.embed_dict_master, 
                                            new_embed_dict_master['wordsim'], 
                                            evalD.wordsim_df, "SVC_Wordsim",
                                            comb_mode = False, SVC_or_SVR = 'SVC')
print("Analysed SVC_Wordsim")

SVMProcedures.execute_all_supervised_scenarios(inp.emb_list, basis.basis_list, inp.embed_dict_master, 
                                            new_embed_dict_master['wordsim'], 
                                            evalD.wordsim_df, "SVR_Wordsim",
                                            comb_mode = False, SVC_or_SVR = 'SVR')
print("Analysed SVR_Wordsim")

# SVMProcedures.execute_all_supervised_scenarios(inp.emb_list, basis.basis_list, inp.embed_dict_master, 
#                                             new_embed_dict_master['wordsim'], 
#                                             evalD.wordsim_df, "SVR_Wordsim",
#                                             comb_mode = True, SVC_or_SVR = 'SVR')
# print("Analysed SVR_Wordsim combinatrics")

new_embed_dict_master, responses_dict_master = {}, {}

# Wiki CS executions
new_embed_dict_master['wiki_cs'], responses_dict_master['wiki_cs'] = RetrofittingProcedures.execute_all_unsupervised_scenarios(inp.emb_list, basis.basis_list, inp.embed_dict_master, basis.neighbors_dict_master, 
                                                          evalD.wiki_cs_df, "wiki_cs_ind")
print("Analysed wiki_cs_ind")

SVMProcedures.execute_all_supervised_scenarios(inp.emb_list, basis.basis_list, inp.embed_dict_master,
                                            new_embed_dict_master['wiki_cs'],
                                            evalD.wordsim_df, "SVC_Wiki_CS",
                                            comb_mode = False, SVC_or_SVR = 'SVC')
print("Analysed SVC_Wiki_CS
      0.
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      
      ")

# Conceptnet executions
new_embed_dict_master['conceptnet'], responses_dict_master['conceptnet'] = RetrofittingProcedures.execute_all_unsupervised_scenarios(inp.emb_list, basis.basis_list, inp.embed_dict_master, basis.neighbors_dict_master, 
                                                          evalD.concept_net_df, "concept_net_ind")
print("Analysed concept_net_ind")

SVMProcedures.execute_all_supervised_scenarios(inp.emb_list, basis.basis_list, inp.embed_dict_master,
                                            new_embed_dict_master['conceptnet'],
                                            evalD.wordsim_df, "SVC_Conceptnet",
                                            comb_mode = False, SVC_or_SVR = 'SVC')
print("Analysed SVC_Conceptnet")

print(f"Time taken for end-to-end execution: {time() - start_master_time}s")

Neighbor Datasets:   0%|          | 0/9 [00:00<?, ?it/s]

Fetched neighbour datasets: ['bert_child_par', 'bert_siblings', 'bert_all', 'cross_enc_child_par', 'cross_enc_siblings', 'cross_enc_all', 'class_child_par', 'class_siblings', 'class_all']
Dataset: Wordsim-353
M    221
U    103
I     20
Name: category, dtype: int64
Dataset: Wikidata CS
M    100110
I      1415
Name: category, dtype: int64
Dataset: Concept Net
M    10220
I     2175
Name: category, dtype: int64


Basis:   0%|          | 0/9 [00:00<?, ?it/s]

Embedding:   0%|          | 0/9 [00:00<?, ?it/s]

Weight Case:   0%|          | 0/2 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/3 [00:00<?, ?it/s]

Weight Case:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Evaluation Datasets

In [3]:
bioDF = pd.read_csv('../data/pedersen2007measures_table1.csv')

In [7]:
bioDF.head()

,Term1,Term1_kg_id,Term1_kg_label,Term1_description,Term2,Term2_kg_id,Term2_kg_label,Term2_description,Physician,Coder
0,Renal failure,Q476921,kidney failure,disease where the kidneys fail to adequately f...,Kidney failure,Q476921,kidney failure,disease where the kidneys fail to adequately f...,4.0,4.0
1,Heart,Q1072,heart,organ for the circulation of blood in animal c...,Myocardium,Q84133,myocardium,"middle layer of the heart wall, which consists...",3.3,3.0
2,Stroke,Q12202,stroke,problem with the arteries supplying blood to t...,Infarct,Q207550,infarction,"tissue death caused by a local lack of oxygen,...",3.0,2.8
3,Abortion,Q8452,abortion,intentional ending of a pregnancy,Miscarriage,Q28693,miscarriage,natural death of an embryo or fetus before it ...,3.0,3.3
4,Delusion,Q189643,delusion,firm and fixed belief based on inadequate grou...,Schizophrenia,Q41112,schizophrenia,psychotic disorder characterized by emotional ...,3.0,2.2


In [10]:
bioDFNodesSet = set(bioDF.Term1_kg_id.to_list() + bioDF.Term2_kg_id.to_list())

In [11]:
P279childParNodesSet = set(p279WordSimSeededDF_wabs_text.node1.to_list() + p279WordSimSeededDF_wabs_text.node2.to_list())
P279siblingsNodesSet = set(p279Seeded_SiblingsDF3_wabs_text.node1.to_list() + p279Seeded_SiblingsDF3_wabs_text.node2.to_list())

In [14]:
sum(bioDF.Term1_kg_id.apply(lambda p: p in P279childParNodesSet or p in P279siblingsNodesSet)), \
sum(bioDF.Term2_kg_id.apply(lambda p: p in P279childParNodesSet or p in P279siblingsNodesSet))

(2, 1)

In [20]:
probaseNodesSet = set(probDF_Qnodes_DF_WQnodes1_subset.node1.to_list() + probDF_Qnodes_DF_WQnodes1_subset.node2.to_list())

sum(bioDF.Term1_kg_id.apply(lambda p: p in probaseNodesSet)), \
sum(bioDF.Term2_kg_id.apply(lambda p: p in probaseNodesSet))

(23, 25)

# Dataset-Target Combinations

## Retrofitting

### Retrofitting using concatenation of embeddings

In [26]:
from collections import defaultdict
def generateConcatenatedEmbeddingDict(embedDictMaster, key_comb, embedding_lengths):
    embedDict = defaultdict(list)
    masterKeySet = set()
    for key in key_comb:
        for qnode in embedDictMaster[key]:
            masterKeySet.add(qnode)
    for qnode in masterKeySet:
        for key in key_comb:
            if qnode in embedDictMaster[key]:
                embedDict[qnode] = embedDict[qnode] + (embedDictMaster[key][qnode].tolist())
            else:
                embedDict[qnode] = embedDict[qnode] + [0]*embedding_lengths[key]
        embedDict[qnode] = np.array(embedDict[qnode])
    return dict(embedDict)

In [27]:
%%time
embedDictMaster['text_7-text_2-comp-tran-abs-abs_first'] = generateConcatenatedEmbeddingDict(embedDictMaster, 
                ['text_7_props', 'text_2_props', 'complex', 'transe', 'abstract', 'abstract_first_sent'], 
                embedding_lengths)

CPU times: user 1min 16s, sys: 5.4 s, total: 1min 21s
Wall time: 1min 21s


In [ ]:
%%time
newEmbedDictMaster = {}
responsesDictMaster = {}

results = []
NUM_ITERS = 2
for basis in tqdm(basisList):
    for weightedNess in [True]:
        for i in tqdm(range(2,len(embList)+1)):
            for embComb in tqdm(combinations(embList, i)):
                emb = '-'.join(embComb)
                start_time = time()
                embedDict = {}
                embedDict[1] = embedDict[2] = generateConcatenatedEmbeddingDict(embedDictMaster,embComb, embedding_lengths)
                
                for weightCase in [1,2]:
                    oldAcc = 0
#                     groupResults = []
                    for iterNum in range(1,NUM_ITERS+1):
                        if weightedNess:
                            caseName = emb + '_' + basis + '_' + str(weightCase) + '_weighted'
                        else:
                            caseName = emb + '_' + basis + '_' + str(weightCase) + '_unweighted'
                        newEmbedDict = retrofit(embedDict[weightCase], neighDictMaster[basis], weightCase, weightedNess)
            #             dists = determine_distances(embedDict, newEmbedDict)
                        responsesDict = fetchCorrelationResults(embedDict[weightCase], newEmbedDict)
            #                 print(responsesDict.keys())
                        if iterNum == 1:
                            oldAcc = responsesDict['old_acc']*100
                            groupResults.append([emb, basis, weightCase, weightedNess, iterNum, \
                                        -1, \
                                        responsesDict['old_acc']*100, \
                                        0, \
                                        responsesDict['coveredPairs'], \
                                         responsesDict['class_rep_old']['I']['precision'], \
                                         responsesDict['class_rep_old']['I']['recall'], \
                                         responsesDict['class_rep_old']['I']['f1-score'], \
                                         responsesDict['class_rep_old']['U']['precision'], \
                                         responsesDict['class_rep_old']['U']['recall'], \
                                         responsesDict['class_rep_old']['U']['f1-score'], \
                                         responsesDict['class_rep_new']['I']['precision'], \
                                         responsesDict['class_rep_new']['I']['recall'], \
                                         responsesDict['class_rep_new']['I']['f1-score'], \
                                         responsesDict['class_rep_new']['U']['precision'], \
                                         responsesDict['class_rep_new']['U']['recall'], \
                                         responsesDict['class_rep_new']['U']['f1-score'], \
                                         i, \
                                         time() - start_time
                                        ])
                        results.append([emb, basis, weightCase, weightedNess, iterNum, \
                                        -1, \
                                        responsesDict['new_acc']*100, \
                                        (responsesDict['new_acc'] - oldAcc)*100, \
                                        responsesDict['coveredPairs'], \
                                         responsesDict['class_rep_old']['I']['precision'], \
                                         responsesDict['class_rep_old']['I']['recall'], \
                                         responsesDict['class_rep_old']['I']['f1-score'], \
                                         responsesDict['class_rep_old']['U']['precision'], \
                                         responsesDict['class_rep_old']['U']['recall'], \
                                         responsesDict['class_rep_old']['U']['f1-score'], \
                                         responsesDict['class_rep_new']['I']['precision'], \
                                         responsesDict['class_rep_new']['I']['recall'], \
                                         responsesDict['class_rep_new']['I']['f1-score'], \
                                         responsesDict['class_rep_new']['U']['precision'], \
                                         responsesDict['class_rep_new']['U']['recall'], \
                                         responsesDict['class_rep_new']['U']['f1-score'], \
                                         i, \
                                         time() - start_time
                                        ])
                        embedDict[weightCase] = newEmbedDict

                        newEmbedDictMaster[caseName] = newEmbedDict
                        responsesDictMaster[caseName] = responsesDict
                        if responsesDict['new_acc']*100 < oldAcc:
                            break
#                 results.append(gr)
#                     highestOne = True
#                     for gR, rank in zip(groupResults, np.argsort([-p[6] for p in groupResults])):
#                         results.append(gR+[rank])
#                     if iterNum == NUM_ITERS and highestOne:
#                         caseName = gR[0] + '_' + gR[1] + '_' + str(gR[2]) + '_weighted'
#                         newEmbedDictMaster[caseName] = serializeEmbeddingDict(newEmbedDictMaster[caseName])
#                         highestOne = False
#                         json.dump(newEmbedDictMaster[caseName],open('../data/Master_P279_dataset/embeddings/new_embedding_dict_'+caseName+'.json','w'))
#                         newEmbedDictMaster[caseName] = deserializeEmbeddingDict(newEmbedDictMaster[caseName])
resultsDF = pd.DataFrame(results, columns=['Embedding', 'Basis', 'Weight', 'Weightedness', 'Iteration Num', 'Old Acc', 'New Acc', 'Increase', 'Pairs Covered', \
                                           'Old I Precision', 'Old I Recall', 'Old I F1-Score', \
                                           'Old U Precision', 'Old U Recall', 'Old U F1-Score', \
                                           'New I Precision', 'New I Recall', 'New I F1-Score', \
                                           'New U Precision', 'New U Recall', 'New U F1-Score', \
                                           'No. of embeddings', \
                                           'Time'])
resultsDF.to_csv('../data/retrofitting/retro_concat_'+ today_date +'.csv', index=False)


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

## SVM

In [64]:
svmCasesList = []
for basis in tqdm(basisList):
    for emb in embList:
        for weightedNess in [True]:
            for iterNum in range(0,NUM_ITERS+1):
                for weightCase in [1,2]:
                    for svm_input in ['emb','score']:
                        temp_dict = {'basis': basis, 'emb': emb, 'weightedNess': weightedNess, 
                                    'iterNum': iterNum, 'weightCase': weightCase, 'svm_input': svm_input}
                        svmCasesList.append(temp_dict)

  0%|          | 0/6 [00:00<?, ?it/s]

### SVM Classification

In [128]:
pd.DataFrame(results, columns=['caseName','basis','emb','weightedNess', 'iteration', 'weightCase', 'technique','accuracy']).to_csv('../data/retrofitting/retro_SVM_test_'+ today_date +'.csv', index=False)

In [129]:
def trainAndFindAccuracyCombs(wordSim353AnnotDF_New_Merged_DF, case):
    X = []
    
    if case['svm_input'] == 'emb':
        for _, row in wordSim353AnnotDF_New_Merged_DF.iterrows():
            tempX = []
            for emb in case['emb']:
                caseName = emb + '_' + case['basis'] + '_' + str(case['weightCase']) + ('_weighted' if case['weightedNess'] else '_unweighted')
                # Guard check conditions
                if case['iterNum'] != 0 and caseName not in newEmbedDictMaster:
                    return caseName, case, None
                if case['iterNum'] == 0:
                    tempX += embedDictMaster[emb][row['word1_kg_id']].tolist() + embedDictMaster[emb][row['word2_kg_id']].tolist()
                else:
                    tempX += newEmbedDictMaster[caseName][row['word1_kg_id']].tolist() + newEmbedDictMaster[caseName][row['word2_kg_id']].tolist()
            X.append(tempX)
    else:
        for _, row in wordSim353AnnotDF_New_Merged_DF.iterrows():
            tempX = []
            for emb in case['emb']:
                caseName = emb + '_' + case['basis'] + '_' + str(case['weightCase']) + ('_weighted' if case['weightedNess'] else '_unweighted')
                # Guard check conditions
                if case['iterNum'] != 0 and caseName not in newEmbedDictMaster:
                    return caseName, case, None
                if case['iterNum'] == 0:
                    tempX += [(abs(cosine_similarity(embedDictMaster[emb][row['word1_kg_id']].reshape(1,-1), embedDictMaster[emb][row['word2_kg_id']].reshape(1,-1))[0][0]))]
                else:
                    tempX += [(abs(cosine_similarity(newEmbedDictMaster[caseName][row['word1_kg_id']].reshape(1,-1), newEmbedDictMaster[caseName][row['word2_kg_id']].reshape(1,-1))[0][0]))]
            X.append(tempX)
#     X = wordSim353AnnotDF_New_Merged_DF[colList]
    X = pd.DataFrame(X)
    Y = wordSim353AnnotDF_New_Merged_DF['category']

    N_SPLITS = 10

    skf = StratifiedKFold(n_splits=N_SPLITS, random_state=19, shuffle=True)
    X_train_splits, X_test_splits, Y_train_splits, Y_test_splits = [], [], [], []
    for train_index, test_index in skf.split(X, Y):
        X_train_splits.append(X.iloc[train_index])
        X_test_splits.append(X.iloc[test_index])
        Y_train_splits.append(Y.iloc[train_index])
        Y_test_splits.append(Y.iloc[test_index])

    preds = []
    for X_train1, Y_train1, X_test1, Y_test1 in zip(X_train_splits, Y_train_splits, X_test_splits, Y_test_splits):
        clf = make_pipeline(StandardScaler(), SVC(gamma='auto', random_state=100))
        clf.fit(X_train1, Y_train1)
        preds.append(clf.predict(X_test1))

    tempVals = []

    acc = 0
    for pred, Y_test1 in zip(preds, Y_test_splits):
        acc += accuracy_score(pred, Y_test1)

    return caseName, *list(case.values()), acc/N_SPLITS

In [23]:
svmCombCasesList = []
for basis in tqdm(basisList):
        for weightedNess in [True]:
            for iterNum in range(0,NUM_ITERS+1):
                for weightCase in [1,2]:
                    for svm_input in ['score']:
                        for i in range(1,len(embList)+1):
                            for embComb in combinations(embList, i):
                                temp_dict = {'basis': basis, 'emb': embComb, 'weightedNess': weightedNess, 
                                            'iterNum': iterNum, 'weightCase': weightCase, 'svm_input': svm_input}
                                svmCombCasesList.append(temp_dict)

NameError: name 'basisList' is not defined

In [138]:
len(svmCombCasesList)

3024

In [142]:
%%time
from joblib import Parallel, delayed

wordSim353AnnotDF_New = pd.read_csv('../data/wordsim353_with_r3.csv')
    
results = Parallel(n_jobs=1)(delayed(trainAndFindAccuracyCombs)(wordSim353AnnotDF_New, caseDict,) for caseDict in tqdm(svmCombCasesList))
resDF = pd.DataFrame(results, columns=['caseName','basis','emb_comb','weightedNess', 'iteration', 'weightCase', 'technique','accuracy'])
resDF['emb_comb'] = resDF['emb_comb'].apply(lambda p: " & ".join(p))
resDF.to_csv('../data/retrofitting/retro_SVM_comb_test_'+ today_date +'.csv', index=False)
# print(results)

  0%|          | 0/3024 [00:00<?, ?it/s]

CPU times: user 18min 7s, sys: 24.6 s, total: 18min 32s
Wall time: 18min 27s


### SVM Regression

In [82]:
from joblib import Parallel, delayed

wordSim353AnnotDF_New = pd.read_csv('../data/wordsim353_with_r3.csv')
    
results = Parallel(n_jobs=1)(delayed(trainAndFindRegressionAccuracy)(wordSim353AnnotDF_New, caseDict,) for caseDict in tqdm(svmCasesList))
pd.DataFrame(results, columns=['caseName','basis','emb','weightedNess', 'iteration', 'weightCase', 'technique','Acc']).to_csv('../data/retrofitting/retro_SVM_Reg_test_'+ today_date +'.csv', index=False)
# print(results)

  0%|          | 0/432 [00:00<?, ?it/s]

In [84]:
def trainAndFindRegressionAccuracyCombs(wordSim353AnnotDF_New_Merged_DF, case):
    X = []
    
    if case['svm_input'] == 'emb':
        for _, row in wordSim353AnnotDF_New_Merged_DF.iterrows():
            tempX = []
            for emb in case['emb']:
                caseName = emb + '_' + case['basis'] + '_' + str(case['weightCase']) + ('_weighted' if case['weightedNess'] else '_unweighted')
                # Guard check conditions
                if case['iterNum'] != 0 and caseName not in newEmbedDictMaster:
                    return caseName, case, None
                if case['iterNum'] == 0:
                    tempX += embedDictMaster[emb][row['word1_kg_id']].tolist() + embedDictMaster[emb][row['word2_kg_id']].tolist()
                else:
                    tempX += newEmbedDictMaster[caseName][row['word1_kg_id']].tolist() + newEmbedDictMaster[caseName][row['word2_kg_id']].tolist()
            X.append(tempX)
    else:
        for _, row in wordSim353AnnotDF_New_Merged_DF.iterrows():
            tempX = []
            for emb in case['emb']:
                caseName = emb + '_' + case['basis'] + '_' + str(case['weightCase']) + ('_weighted' if case['weightedNess'] else '_unweighted')
                # Guard check conditions
                if case['iterNum'] != 0 and caseName not in newEmbedDictMaster:
                    return caseName, case, None
                if case['iterNum'] == 0:
                    tempX += [(abs(cosine_similarity(embedDictMaster[emb][row['word1_kg_id']].reshape(1,-1), embedDictMaster[emb][row['word2_kg_id']].reshape(1,-1))[0][0]))]
                else:
                    tempX += [(abs(cosine_similarity(newEmbedDictMaster[caseName][row['word1_kg_id']].reshape(1,-1), newEmbedDictMaster[caseName][row['word2_kg_id']].reshape(1,-1))[0][0]))]
            X.append(tempX)
#     X = wordSim353AnnotDF_New_Merged_DF[colList]
    X = pd.DataFrame(X)
    Y = (wordSim353AnnotDF_New_Merged_DF['Avg'] - 1) / 3

    N_SPLITS = 10

    skf = KFold(n_splits=N_SPLITS, random_state=19, shuffle=True)
    X_train_splits, X_test_splits, Y_train_splits, Y_test_splits = [], [], [], []
    for train_index, test_index in skf.split(X, Y):
        X_train_splits.append(X.iloc[train_index])
        X_test_splits.append(X.iloc[test_index])
        Y_train_splits.append(Y.iloc[train_index])
        Y_test_splits.append(Y.iloc[test_index])

    preds = []
    for X_train1, Y_train1, X_test1, Y_test1 in zip(X_train_splits, Y_train_splits, X_test_splits, Y_test_splits):
        clf = make_pipeline(StandardScaler(), SVR(gamma='auto'))
        clf.fit(X_train1, Y_train1)
        preds.append(clf.predict(X_test1))

    tempVals = []

    acc = 0
    for pred, Y_test1 in zip(preds, Y_test_splits):
        acc += accuracy_score(np_label_samples(pred * 3 + 1), np_label_samples(Y_test1 * 3 + 1))
#         acc += mean_squared_error(pred * 3 + 1, Y_test1 * 3 + 1)

    return caseName, *list(case.values()), acc/N_SPLITS

In [87]:
%%time
from joblib import Parallel, delayed

wordSim353AnnotDF_New = pd.read_csv('../data/wordsim353_with_r3.csv')
    
results = Parallel(n_jobs=1)(delayed(trainAndFindRegressionAccuracyCombs)(wordSim353AnnotDF_New, caseDict,) for caseDict in tqdm(svmCombCasesList))
resDF = pd.DataFrame(results, columns=['caseName','basis','emb_comb','weightedNess', 'iteration', 'weightCase', 'technique','accuracy'])
resDF['emb_comb'] = resDF['emb_comb'].apply(lambda p: " & ".join(p))
resDF.to_csv('../data/retrofitting/retro_SVM_Reg_comb_test_'+ today_date +'.csv', index=False)

  0%|          | 0/3024 [00:00<?, ?it/s]

CPU times: user 17min 14s, sys: 18.4 s, total: 17min 32s
Wall time: 17min 26s


# Determine Differences

In [37]:
wordSim353AnnotDF_New = pd.read_csv('../data/wordsim353_with_r3.csv')

In [39]:
oldEmbedDictKey = 'transe'
newEmbedDictKey = 'transe_19k_childPar_1_weighted'
wordSim353AnnotDF_New['textOld'] = wordSim353AnnotDF_New.apply(lambda p: 4 - 3 * abs(cosine_similarity(np.array(embedDictMaster[oldEmbedDictKey][p['word1_kg_id']]).reshape(1,-1), np.array(embedDictMaster[oldEmbedDictKey][p['word2_kg_id']]).reshape(1,-1))[0][0]) if p['word1_kg_id'] in embedDictMaster[oldEmbedDictKey] and p['word2_kg_id'] in embedDictMaster[oldEmbedDictKey] else -1, axis=1)
wordSim353AnnotDF_New['textNew'] = wordSim353AnnotDF_New.apply(lambda p: 4 - 3 * abs(cosine_similarity(np.array(newEmbedDictMaster[newEmbedDictKey][p['word1_kg_id']]).reshape(1,-1), np.array(newEmbedDictMaster[newEmbedDictKey][p['word2_kg_id']]).reshape(1,-1))[0][0]) if p['word1_kg_id'] in embedDictMaster[oldEmbedDictKey] and p['word2_kg_id'] in embedDictMaster[oldEmbedDictKey] else -1, axis=1)

In [41]:
wordSim353AnnotDF_New['old_diff'] = abs(wordSim353AnnotDF_New['textOld'] - wordSim353AnnotDF_New['Avg'])
wordSim353AnnotDF_New['new_diff'] = abs(wordSim353AnnotDF_New['textNew'] - wordSim353AnnotDF_New['Avg'])

In [6]:
pd.options.display.max_colwidth = 50

In [42]:
wordSim353AnnotDF_New.sort_values(by=['old_diff'],ascending=False)['old_diff'].describe()

count    344.000000
mean       0.506079
std        0.388162
min        0.000000
25%        0.201136
50%        0.430653
75%        0.744341
max        1.969069
Name: old_diff, dtype: float64

In [43]:
wordSim353AnnotDF_New.sort_values(by=['new_diff'],ascending=False)['new_diff'].describe()

count    344.000000
mean       0.587435
std        0.440362
min        0.000000
25%        0.237281
50%        0.494043
75%        0.880977
max        1.900974
Name: new_diff, dtype: float64

In [45]:
def determineFarNessOfOld(val1):
    if 0.000000 <= val1 < 0.506079:
        return 0
    else:
        return 1
def determineFarNessOfNew(val1):
    if 0 <= val1 < 0.587435:
        return 0
    else:
        return 1

wordSim353AnnotDF_New['Old_Diff_Cat'] = wordSim353AnnotDF_New['old_diff'].apply(determineFarNessOfOld)
wordSim353AnnotDF_New['New_Diff_Cat'] = wordSim353AnnotDF_New['new_diff'].apply(determineFarNessOfNew)
wordSim353AnnotDF_New['correct_category'] = wordSim353AnnotDF_New.Avg.apply(label_samples)
wordSim353AnnotDF_New['old_predicted_category'] = wordSim353AnnotDF_New.textOld.apply(label_samples)
wordSim353AnnotDF_New['new_predicted_category'] = wordSim353AnnotDF_New.textNew.apply(label_samples)

In [49]:
wordSim353AnnotDF_New_Merged_DF = wordSim353AnnotDF_New[['Word 1', 'Word 2', 'Avg', 'textOld', 'textNew', 'old_diff', 'new_diff', 'Old_Diff_Cat', 'New_Diff_Cat', 'correct_category', 'old_predicted_category', 'new_predicted_category']]

In [50]:
wordSim353AnnotDF_New_Merged_DF

,Word 1,Word 2,Avg,textOld,textNew,old_diff,new_diff,Old_Diff_Cat,New_Diff_Cat,correct_category,old_predicted_category,new_predicted_category
0,Arafat,peace,3.60,3.909414,3.837159,0.309414,0.237159,0,0,U,U,U
1,Arafat,terror,3.60,3.646852,3.593611,0.046852,0.006389,0,0,U,U,U
2,FBI,fingerprint,3.60,3.470644,3.470644,0.129356,0.129356,0,0,U,M,M
3,FBI,investigation,3.00,3.674395,3.625026,0.674395,0.625026,1,1,M,U,U
4,Harvard,Yale,2.20,2.514591,2.514591,0.314591,0.314591,0,0,M,M,M
...,...,...,...,...,...,...,...,...,...,...,...,...
339,war,troops,3.00,2.877362,2.476802,0.122638,0.523198,0,0,M,M,M
340,weapon,secret,3.80,3.544676,3.078550,0.255324,0.721450,0,1,U,U,M
341,weather,forecast,3.00,2.443100,2.075806,0.556900,0.924194,1,1,M,M,M
342,wood,forest,1.80,3.666691,3.692986,1.866691,1.892986,1,1,M,U,U


In [12]:
len(wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['Old_Diff_Cat'] == 0) & (wordSim353AnnotDF_New_Merged_DF['New_Diff_Cat'] == 0)])

123

In [51]:
wordSim353AnnotDF_New_Merged_DF[wordSim353AnnotDF_New_Merged_DF['old_predicted_category'] != wordSim353AnnotDF_New_Merged_DF['correct_category']].to_csv('../data/transEmb/badBefore_'+ today_date +'.csv',index=None)

In [52]:
wordSim353AnnotDF_New_Merged_DF[wordSim353AnnotDF_New_Merged_DF['new_predicted_category'] != wordSim353AnnotDF_New_Merged_DF['correct_category']].to_csv('../data/transEmb/badAfter_'+ today_date +'.csv',index=None)

In [28]:
wordSim353AnnotDF_New_Merged_DF.to_csv('../data/transEmb/entireSet.csv',index=None)

In [58]:
wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['Old_Diff_Cat'] == 0) & (wordSim353AnnotDF_New_Merged_DF['New_Diff_Cat'] == 0)].to_csv('../data/transEmb/good+good_'+ today_date +'.csv',index=None)

In [59]:
wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['Old_Diff_Cat'] == 0) & (wordSim353AnnotDF_New_Merged_DF['New_Diff_Cat'] == 1)].to_csv('../data/transEmb/good+bad_'+ today_date +'.csv',index=None)

In [60]:
wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['Old_Diff_Cat'] == 1) & (wordSim353AnnotDF_New_Merged_DF['New_Diff_Cat'] == 0)].to_csv('../data/transEmb/bad+good_'+ today_date +'.csv',index=None)

In [61]:
wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['Old_Diff_Cat'] == 1) & (wordSim353AnnotDF_New_Merged_DF['New_Diff_Cat'] == 1)].to_csv('../data/transEmb/bad+bad_'+ today_date +'.csv',index=None)

In [4]:
wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_old_cosSim_Diff'] = abs(wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_old_cosSim'] - wordSim353AnnotDF_New_Merged_DF['Avg'])

In [5]:
wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_2_new_cosSim_Diff'] = abs(wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_2_new_cosSim'] - wordSim353AnnotDF_New_Merged_DF['Avg'])

In [6]:
pd.options.display.max_colwidth = 50

In [8]:
wordSim353AnnotDF_New_Merged_DF.sort_values(by=['concat_19k_v1_old_cosSim_Diff'],ascending=False)['concat_19k_v1_old_cosSim_Diff'].describe()

count    344.000000
mean       0.567297
std        0.435379
min        0.000000
25%        0.250130
50%        0.508725
75%        0.835197
max        5.000000
Name: concat_19k_v1_old_cosSim_Diff, dtype: float64

In [9]:
wordSim353AnnotDF_New_Merged_DF.sort_values(by=['concat_19k_v1_2_new_cosSim_Diff'],ascending=False)['concat_19k_v1_2_new_cosSim_Diff'].describe()

count    3.440000e+02
mean     5.938081e-01
std      4.503200e-01
min      4.440892e-16
25%      2.665817e-01
50%      5.349785e-01
75%      8.528840e-01
max      5.000000e+00
Name: concat_19k_v1_2_new_cosSim_Diff, dtype: float64

In [10]:
def determineFarNessOfOld(val1):
    if 0.000000 <= val1 < 0.508725:
        return 0
    else:
        return 1
def determineFarNessOfNew(val1):
    if 0 <= val1 < 5.349785e-01:
        return 0
    else:
        return 1

wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_old_cosSim_Diff_Cat'] = wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_old_cosSim_Diff'].apply(determineFarNessOfOld)
wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_2_new_cosSim_Diff_Cat'] = wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_2_new_cosSim_Diff'].apply(determineFarNessOfNew)

In [11]:
wordSim353AnnotDF_New_Merged_DF = wordSim353AnnotDF_New_Merged_DF[['Word 1', 'Word 2', 'Avg', 'concat_19k_v1_old_cosSim', 'concat_19k_v1_2_new_cosSim', 'concat_19k_v1_old_cosSim_Diff', 'concat_19k_v1_2_new_cosSim_Diff', 'concat_19k_v1_old_cosSim_Diff_Cat', 'concat_19k_v1_2_new_cosSim_Diff_Cat']]


In [12]:
wordSim353AnnotDF_New_Merged_DF

,Word 1,Word 2,Avg,concat_19k_v1_old_cosSim,concat_19k_v1_2_new_cosSim,concat_19k_v1_old_cosSim_Diff,concat_19k_v1_2_new_cosSim_Diff,concat_19k_v1_old_cosSim_Diff_Cat,concat_19k_v1_2_new_cosSim_Diff_Cat
0,Arafat,peace,3.60,3.251198,3.193271,0.348802,0.406729,0,0
1,Arafat,terror,3.60,3.231409,3.157229,0.368591,0.442771,0,0
2,FBI,fingerprint,3.60,2.948487,2.853048,0.651513,0.746952,1,1
3,FBI,investigation,3.00,2.930365,2.895604,0.069635,0.104396,0,0
4,Harvard,Yale,2.20,1.991054,1.991054,0.208946,0.208946,0,0
...,...,...,...,...,...,...,...,...,...
339,war,troops,3.00,2.913931,2.787213,0.086069,0.212787,0,0
340,weapon,secret,3.80,3.152513,3.119065,0.647487,0.680935,1,1
341,weather,forecast,3.00,2.492765,2.462389,0.507235,0.537611,0,1
342,wood,forest,1.80,2.612681,2.606941,0.812681,0.806941,1,1


In [13]:
len(wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_old_cosSim_Diff_Cat'] == 0) & (wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_2_new_cosSim_Diff_Cat'] == 0)])

164

In [15]:
wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_old_cosSim_Diff_Cat'] == 0) & (wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_2_new_cosSim_Diff_Cat'] == 0)].to_csv('../data/concEmb/good+good.csv',index=None)

In [16]:
wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_old_cosSim_Diff_Cat'] == 0) & (wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_2_new_cosSim_Diff_Cat'] == 1)].to_csv('../data/concEmb/good+bad.csv',index=None)

In [17]:
wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_old_cosSim_Diff_Cat'] == 1) & (wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_2_new_cosSim_Diff_Cat'] == 0)].to_csv('../data/concEmb/bad+good.csv',index=None)

In [18]:
wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_old_cosSim_Diff_Cat'] == 1) & (wordSim353AnnotDF_New_Merged_DF['concat_19k_v1_2_new_cosSim_Diff_Cat'] == 1)].to_csv('../data/concEmb/bad+bad.csv',index=None)

# SVM Models

In [144]:
!pip install datacompy

In [177]:
wordSim353AnnotDF_New_Merged_DF = pd.read_csv('../data/wordsim353_all_embeddings_with_retrofits_new.csv')

In [181]:
wordSim353AnnotDF_New_Merged_DF_old = pd.read_csv('../data/wordsim353_all_embeddings_with_retrofits.csv')

In [142]:
wordSim353AnnotDF_New_Merged_DF_old.head()

,Word 1,Word 2,ID,H_Sim,H_Dim,F_Sim,F_Dim,N_Sim,N_Dim,D_Sim,...,concat_19k_v2_word1_new,concat_19k_v2_word2_new,concat_19k_v2_old_cosSim,concat_19k_v2_2_new_cosSim,concat_probase_19k_v2_word1_old,concat_probase_19k_v2_word2_old,concat_probase_19k_v2_word1_new,concat_probase_19k_v2_word2_new,concat_probase_19k_v2_old_cosSim,concat_probase_19k_v2_2_new_cosSim
0,Arafat,peace,8,3,D,4,NaN,3,U,4,...,"[0.006720443092017323, 0.008331487679612525, 0...","[-0.004515859576248157, -0.01172366887295375, ...",3.244539,3.175157,"[0.006720440291834372, 0.008331484208161396, 0...","[-0.005011610937734838, -0.013383541550578623,...","[0.006720440291834372, 0.008331484208161396, 0...","[-0.00497129831746384, -0.01325426749683971, -...",3.244539,3.240270
1,Arafat,terror,9,3,D,4,NaN,3,U,4,...,"[0.006720443092017323, 0.008331487679612525, 0...","[-0.004862257088069111, 0.014077170590128979, ...",3.085083,3.013958,"[0.006720440291834372, 0.008331484208161396, 0...","[-0.005805280438126139, 0.015115419696749257, ...","[0.006720440291834372, 0.008331484208161396, 0...","[-0.004862255062129871, 0.014077164724644747, ...",3.085083,3.013958
2,FBI,fingerprint,109,3,D,4,NaN,4,NaN,3,...,"[0.006466897059812569, 0.014694837542908411, -...","[0.01523113309590259, 0.005271417380511587, -0...",2.823093,2.745230,"[0.006466894365273728, 0.014694831420063071, -...","[0.013922101136130544, 0.0023911842131386003, ...","[0.006466894365273728, 0.014694831420063071, -...","[0.013814564227839972, 0.0024352598952305004, ...",2.823093,2.816176
3,FBI,investigation,110,3,U,3,U,3,U,3,...,"[0.006466897059812569, 0.014694837542908411, -...","[0.013016090623183913, 0.01114268771757639, -0...",2.920986,2.886161,"[0.006466894365273728, 0.014694831420063071, -...","[0.01327496060589727, 0.010965634890657862, -0...","[0.006466894365273728, 0.014694831420063071, -...","[0.013242882546106787, 0.010953106196386554, -...",2.920986,2.919120
4,Harvard,Yale,137,2,S,3,S,2,S,2,...,"[0.01322626159341165, 0.005047111630696947, -0...","[0.010410829393995813, 0.014776997996557633, -...",1.707655,1.707655,"[0.013226256082472592, 0.005047109527735016, -...","[0.010410825056152807, 0.014776991839478788, -...","[0.013226256082472592, 0.005047109527735016, -...","[0.010410825056152807, 0.014776991839478788, -...",1.707655,1.707655


In [184]:
wordSim353AnnotDF_New_Merged_DF[list(filter(lambda p: 'cosSim' in p, wordSim353AnnotDF_New_Merged_DF.columns.to_list()))].describe()

,text_7props_19k_old_cosSim,text_7props_19k_2_new_cosSim,text_2props_19k_old_cosSim,text_2props_19k_2_new_cosSim,complex_19k_old_cosSim,complex_19k_2_new_cosSim,transe_19k_old_cosSim,transe_19k_1_new_cosSim,abstract_19k_old_cosSim,abstract_19k_2_new_cosSim,...,transe_probase_19k_old_cosSim,transe_probase_19k_1_new_cosSim,concat_19k_v1_old_cosSim,concat_19k_v1_2_new_cosSim,concat_probase_19k_v1_old_cosSim,concat_probase_19k_v1_2_new_cosSim,concat_19k_v2_old_cosSim,concat_19k_v2_2_new_cosSim,concat_probase_19k_v2_old_cosSim,concat_probase_19k_v2_2_new_cosSim
count,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,...,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000,344.000000
mean,2.511033,2.464876,2.482254,2.435161,2.983302,2.684818,2.983302,2.680002,2.652866,2.593916,...,2.983302,2.565886,2.875381,2.826493,2.875381,2.865101,2.868066,2.816685,2.868066,2.856603
std,1.148821,1.152911,1.151060,1.155427,0.645911,0.539092,0.645911,0.619239,0.680525,0.653512,...,0.645911,0.596750,0.573902,0.575585,0.573902,0.574410,1.097121,1.106362,1.097121,1.099179
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.005097,1.963229,1.993424,1.949869,2.588591,2.455431,2.588591,2.311818,2.180912,2.137568,...,2.588591,2.183686,2.577938,2.551033,2.577938,2.583811,2.384337,2.350645,2.384337,2.377672
50%,2.328276,2.273674,2.274668,2.232156,3.039087,2.708569,3.039087,2.722278,2.616810,2.591640,...,3.039087,2.608041,2.950578,2.912098,2.950578,2.933984,2.716471,2.657078,2.716471,2.704941
75%,2.636907,2.576155,2.580482,2.527554,3.359243,2.959259,3.359243,3.090563,3.208919,3.060782,...,3.359243,2.935226,3.208735,3.164426,3.208735,3.196270,2.992265,2.899626,2.992265,2.977770
max,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,4.340089,4.327902,...,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000


In [182]:
import datacompy
compare = datacompy.Compare(
    wordSim353AnnotDF_New_Merged_DF[['Word 1',
 'Word 2',
 'Avg',
 'word1_kg_id',
 'word2_kg_id',
 'category'] + list(filter(lambda p: 'cosSim' in p, wordSim353AnnotDF_New_Merged_DF.columns.to_list()))],
    wordSim353AnnotDF_New_Merged_DF_old[['Word 1',
 'Word 2',
 'Avg',
 'word1_kg_id',
 'word2_kg_id',
 'category'] + list(filter(lambda p: 'cosSim' in p, wordSim353AnnotDF_New_Merged_DF_old.columns.to_list()))],
    join_columns = ['Word 1', 'Word 2'])

In [183]:
print(compare.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0       df1       34   344
1       df2       34   344

Column Summary
--------------

Number of columns in common: 32
Number of columns in df1 but not in df2: 2
Number of columns in df2 but not in df1: 2

Row Summary
-----------

Matched on: word 1, word 2
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 344
Number of rows in df1 but not in df2: 0
Number of rows in df2 but not in df1: 0

Number of rows with some compared columns unequal: 343
Number of rows with all compared columns equal: 1

Column Comparison
-----------------

Number of columns compared with some values unequal: 27
Number of columns compared with all values equal: 5
Total number of values which compare unequal: 5,136

Columns with Unequal Values or Types
------------------------------------

                                 Column df1 dtype df2 dtype  # Uneq

In [157]:
wordSim353AnnotDF_New_Merged_DF.head()

,word1_kg_id,word2_kg_id,word 1,word 2,id,h_sim,h_dim,f_sim,f_dim,n_sim,...,concat_19k_v2_word2_new,concat_19k_v2_old_cossim,concat_19k_v2_2_new_cossim,concat_probase_19k_v2_word1_old,concat_probase_19k_v2_word2_old,concat_probase_19k_v2_word1_new,concat_probase_19k_v2_word2_new,concat_probase_19k_v2_old_cossim,concat_probase_19k_v2_2_new_cossim,avg_old
0,Q106106,Q11460,closet,clothes,42,3,H,3,H,3,...,"[0.0014151543049584102, 0.003161483781535701, ...",2.697022,2.652559,"[0.013902607489678993, 0.013241494756981564, -...","[0.0014224480432097992, 0.0031547764732941653,...","[0.01371531560905985, 0.013126875633706597, -0...","[0.0014206775968733409, 0.0031581452887393872,...",2.697022,2.683318,3.0
1,Q107,Q16502,space,world,293,3,L,3,S,3,...,"[-0.0031782974877087693, 0.003973805776938075,...",2.485859,2.347308,"[0.00027019944749503126, 0.005353652289998999,...","[-0.0032792710227188605, 0.0036472880832864947...","[0.00027492382990721006, 0.0053545622809597, -...","[-0.0032450374860671267, 0.0036673577053825254...",2.485859,2.481352,3.2
2,Q107,Q2329,space,chemistry,292,4,NaN,4,NaN,4,...,"[0.015268952851071075, -0.0024137893437008715,...",2.798788,2.782963,"[0.00027019944749503126, 0.005353652289998999,...","[0.015322423107720673, -0.0024500099720118624,...","[0.00027492382990721006, 0.0053545622809597, -...","[0.015281665873653082, -0.002421165402091456, ...",2.798788,2.795047,4.0
3,Q1074,Q7364,skin,eye,288,3,H,3,H,3,...,"[0.002992664951436964, 0.013922942027531155, -...",2.355813,2.314417,"[-0.0026976204618851985, 0.001541600665881322,...","[0.0033640181667058486, 0.014044398230441645, ...","[-0.002678224125439186, 0.0015598357892553907,...","[0.0033484286344994184, 0.014002864550517797, ...",2.355813,2.350603,3.0
4,Q1079023,Q500834,championship,tournament,38,1,NaN,1,NaN,2,...,"[0.00920825730356781, 0.011965374832474578, 0....",1.868699,1.865663,"[-0.0008601077391385219, 0.005201185839222482,...","[0.00923153244996602, 0.011960544533439752, 0....","[-0.0008458279784483333, 0.005209858282773185,...","[0.009207264410626997, 0.011951895965764294, 0...",1.868699,1.866785,1.2


In [107]:
cosine_similarity(np.array([float(val1) for val1 in wordSim353AnnotDF_New_Merged_DF.iloc[0]['concat_19k_v2_word1_new'][1:-1].split(",")]).reshape(1,-1), np.array([float(val1) for val1 in wordSim353AnnotDF_New_Merged_DF.iloc[0]['concat_19k_v2_word2_new'][1:-1].split(",")]).reshape(1,-1))

array([[0.20519776]])

In [108]:
wordSim353AnnotDF_New_Merged_DF.columns

Index(['Word 1', 'Word 2', 'ID', 'H_Sim', 'H_Dim', 'F_Sim', 'F_Dim', 'N_Sim',
       'N_Dim', 'D_Sim',
       ...
       'concat_19k_v2_word1_new', 'concat_19k_v2_word2_new',
       'concat_19k_v2_old_cosSim', 'concat_19k_v2_2_new_cosSim',
       'concat_probase_19k_v2_word1_old', 'concat_probase_19k_v2_word2_old',
       'concat_probase_19k_v2_word1_new', 'concat_probase_19k_v2_word2_new',
       'concat_probase_19k_v2_old_cosSim',
       'concat_probase_19k_v2_2_new_cosSim'],
      dtype='object', length=104)

In [109]:
list(filter(lambda p: 'cosSim' in p, wordSim353AnnotDF_New_Merged_DF.columns))

['text_7props_19k_old_cosSim',
 'text_7props_19k_2_new_cosSim',
 'text_2props_19k_old_cosSim',
 'text_2props_19k_2_new_cosSim',
 'complex_19k_old_cosSim',
 'complex_19k_2_new_cosSim',
 'transe_19k_old_cosSim',
 'transe_19k_1_new_cosSim',
 'abstract_19k_old_cosSim',
 'abstract_19k_2_new_cosSim',
 'abstract_firstSent_19k_old_cosSim',
 'abstract_firstSent_19k_2_new_cosSim',
 'complex_probase_old_cosSim',
 'complex_probase_2_new_cosSim',
 'transe_probase_old_cosSim',
 'transe_probase_1_new_cosSim',
 'complex_probase_19k_old_cosSim',
 'complex_probase_19k_2_new_cosSim',
 'transe_probase_19k_old_cosSim',
 'transe_probase_19k_1_new_cosSim',
 'concat_19k_v1_old_cosSim',
 'concat_19k_v1_2_new_cosSim',
 'concat_probase_19k_v1_old_cosSim',
 'concat_probase_19k_v1_2_new_cosSim',
 'concat_19k_v2_old_cosSim',
 'concat_19k_v2_2_new_cosSim',
 'concat_probase_19k_v2_old_cosSim',
 'concat_probase_19k_v2_2_new_cosSim']

In [110]:
from sklearn.model_selection import StratifiedKFold

import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt 

In [121]:
emb_pairs = [('text_7props_19k', 'text_7props_19k_2'),
             ('text_2props_19k', 'text_2props_19k_2'),
             ('complex_19k', 'complex_19k_2'),
             ('transe_19k', 'transe_19k_1'),
             ('abstract_19k', 'abstract_19k_2'),
             ('abstract_firstSent_19k', 'abstract_firstSent_19k_2'),
             ('complex_probase', 'complex_probase_2'),
             ('transe_probase', 'transe_probase_1'),
             ('complex_probase_19k', 'complex_probase_19k_2'),
             ('transe_probase_19k', 'transe_probase_19k_1'),
             ('concat_19k_v1', 'concat_19k_v1_2'),
             ('concat_probase_19k_v1', 'concat_probase_19k_v1_2'),
             ('concat_19k_v2', 'concat_19k_v2_2'),
             ('concat_probase_19k_v2', 'concat_probase_19k_v2_2')]
colMappers = {p[0]+'_old_cosSim': p[1]+'_new_cosSim' for p in emb_pairs}

In [122]:
oldColList = list(filter(lambda p: 'old_cosSim' in p, wordSim353AnnotDF_New_Merged_DF.columns))

In [123]:
oldColList

['text_7props_19k_old_cosSim',
 'text_2props_19k_old_cosSim',
 'complex_19k_old_cosSim',
 'transe_19k_old_cosSim',
 'abstract_19k_old_cosSim',
 'abstract_firstSent_19k_old_cosSim',
 'complex_probase_old_cosSim',
 'transe_probase_old_cosSim',
 'complex_probase_19k_old_cosSim',
 'transe_probase_19k_old_cosSim',
 'concat_19k_v1_old_cosSim',
 'concat_probase_19k_v1_old_cosSim',
 'concat_19k_v2_old_cosSim',
 'concat_probase_19k_v2_old_cosSim']

In [136]:
wordSim353AnnotDF_New = pd.read_csv('../data/wordsim353_with_r3.csv')

In [185]:
# oldColList, [colMappers[col] for col in oldColList]
t1234 = []
t4567 = []
for col in oldColList:
    t1234.append(accuracy_score(wordSim353AnnotDF_New_Merged_DF.Avg.apply(label_samples), wordSim353AnnotDF_New_Merged_DF[col].apply(label_samples)))
    t4567.append(accuracy_score(wordSim353AnnotDF_New_Merged_DF.Avg.apply(label_samples), wordSim353AnnotDF_New_Merged_DF[colMappers[col]].apply(label_samples)))
                 
                 

In [186]:
t1234 = list(zip(indNames,t1234, t4567))

In [187]:
t1234 = pd.DataFrame(t1234, columns=['Type', 'Accuracy (in %) of old embeddings compared to annotated category', 'Accuracy (in %) of new embeddings compared to annotated category'])

In [188]:
t1234.sort_values(by=['Accuracy (in %) of new embeddings compared to annotated category'], ascending=False)

,Type,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category
10,Complex-Transe-AbsFirstSent - 19k,0.680233,0.686047
11,Complex-Transe-AbsFirstSent - Probase-19k,0.680233,0.680233
8,Complex - Probase-19k,0.598837,0.662791
2,Complex - 19k,0.598837,0.659884
6,Complex - Probase,0.598837,0.656977
3,Transe - 19k,0.598837,0.654070
5,Abstract First Sentence - 19k,0.651163,0.654070
7,Transe - Probase,0.598837,0.645349
9,Transe - Probase-19k,0.598837,0.642442
4,Abstract - 19k,0.627907,0.627907


In [190]:
def trainAndGenerateAccuracies(wordSim353AnnotDF_New_Merged_DF, colList):
    X = wordSim353AnnotDF_New_Merged_DF[colList]
    Y = wordSim353AnnotDF_New_Merged_DF['category']

    N_SPLITS = 10

    skf = StratifiedKFold(n_splits=N_SPLITS, random_state=19, shuffle=True)
    X_train_splits, X_test_splits, Y_train_splits, Y_test_splits = [], [], [], []
    for train_index, test_index in skf.split(X, Y):
        X_train_splits.append(X.iloc[train_index])
        X_test_splits.append(X.iloc[test_index])
        Y_train_splits.append(Y.iloc[train_index])
        Y_test_splits.append(Y.iloc[test_index])

    preds = []
    for X_train1, Y_train1, X_test1, Y_test1 in zip(X_train_splits, Y_train_splits, X_test_splits, Y_test_splits):
        clf = make_pipeline(StandardScaler(), SVC(gamma='auto', random_state=100))
        clf.fit(X_train1, Y_train1)
        preds.append(clf.predict(X_test1))

    tempVals = []

    acc = 0
    for pred, Y_test1 in zip(preds, Y_test_splits):
        acc += accuracy_score(pred, Y_test1)

    tempVals.append(acc/N_SPLITS)

    for col in colList:
        preds = []
        for X_train1, Y_train1, X_test1, Y_test1 in zip(X_train_splits, Y_train_splits, X_test_splits, Y_test_splits):
            clf1 = make_pipeline(StandardScaler(), SVC(gamma='auto', random_state=100))
            clf1.fit(X_train1[[col]], Y_train1)
            preds.append(clf1.predict(X_test1[[col]]))
        acc = 0
        for pred, Y_test1 in zip(preds, Y_test_splits):
            acc += accuracy_score(pred, Y_test1)
        tempVals.append(acc/N_SPLITS)
    return tempVals

def compareEmbeddings(wordSim353AnnotDF_New_Merged_DF, oldColList, newColList, indNames):
    tempVals1 = trainAndGenerateAccuracies(wordSim353AnnotDF_New_Merged_DF, oldColList)
    tempVals2 = trainAndGenerateAccuracies(wordSim353AnnotDF_New_Merged_DF, newColList)
    summ = (pd.DataFrame(list(zip(tempVals1, tempVals2)), index = ['Combined'] + indNames, columns = ['Accuracy (in %) of old embeddings compared to annotated category', 'Accuracy (in %) of new embeddings compared to annotated category']) * 100)
    summ['Increase'] = summ['Accuracy (in %) of new embeddings compared to annotated category'] - summ['Accuracy (in %) of old embeddings compared to annotated category']
    return summ.sort_values(by=['Increase'], ascending=False)


In [59]:
def trainAndFindAccuracy(wordSim353AnnotDF_New_Merged_DF, colList):
    X = wordSim353AnnotDF_New_Merged_DF[colList]
    Y = wordSim353AnnotDF_New_Merged_DF['category']

    N_SPLITS = 10

    skf = StratifiedKFold(n_splits=N_SPLITS, random_state=19, shuffle=True)
    X_train_splits, X_test_splits, Y_train_splits, Y_test_splits = [], [], [], []
    for train_index, test_index in skf.split(X, Y):
        X_train_splits.append(X.iloc[train_index])
        X_test_splits.append(X.iloc[test_index])
        Y_train_splits.append(Y.iloc[train_index])
        Y_test_splits.append(Y.iloc[test_index])

    preds = []
    for X_train1, Y_train1, X_test1, Y_test1 in zip(X_train_splits, Y_train_splits, X_test_splits, Y_test_splits):
        clf = make_pipeline(StandardScaler(), SVC(gamma='auto', random_state=100))
        clf.fit(X_train1, Y_train1)
        preds.append(clf.predict(X_test1))

    tempVals = []

    acc = 0
    for pred, Y_test1 in zip(preds, Y_test_splits):
        acc += accuracy_score(pred, Y_test1)

    return acc/N_SPLITS

In [195]:
def trainAndFindAccuracyRF(wordSim353AnnotDF_New_Merged_DF, colList):
    X = wordSim353AnnotDF_New_Merged_DF[colList]
    Y = wordSim353AnnotDF_New_Merged_DF['category']

    N_SPLITS = 10

    skf = StratifiedKFold(n_splits=N_SPLITS, random_state=19, shuffle=True)
    X_train_splits, X_test_splits, Y_train_splits, Y_test_splits = [], [], [], []
    for train_index, test_index in skf.split(X, Y):
        X_train_splits.append(X.iloc[train_index])
        X_test_splits.append(X.iloc[test_index])
        Y_train_splits.append(Y.iloc[train_index])
        Y_test_splits.append(Y.iloc[test_index])

    preds = []
    for X_train1, Y_train1, X_test1, Y_test1 in zip(X_train_splits, Y_train_splits, X_test_splits, Y_test_splits):
        clf = RandomForestClassifier(max_depth=max(2 * len(colList) // 3,3), random_state=100)
        clf.fit(X_train1, Y_train1)
        preds.append(clf.predict(X_test1))

    tempVals = []

    acc = 0
    for pred, Y_test1 in zip(preds, Y_test_splits):
        acc += accuracy_score(pred, Y_test1)

    return acc/N_SPLITS

In [191]:
oldColList = ['text_7props_19k_old_cosSim',
'text_2props_19k_old_cosSim',
 'complex_19k_old_cosSim',
 'transe_19k_old_cosSim',
 'abstract_19k_old_cosSim',
 'abstract_firstSent_19k_old_cosSim']
indNames = ['Text 7 props - 19k', 'Text 2 props - 19k', 'Complex - 19k', 
            'Transe - 19k', 'Abstract - 19k', 'Abstract First Sentence - 19k']
compareEmbeddings(wordSim353AnnotDF_New_Merged_DF, oldColList, [colMappers[col] for col in oldColList], indNames)

,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase
Text 2 props - 19k,66.000000,67.159664,1.159664
Complex - 19k,66.865546,67.159664,0.294118
Combined,68.899160,69.168067,0.268908
Transe - 19k,66.865546,66.571429,-0.294118
Abstract - 19k,66.546218,65.983193,-0.563025
Text 7 props - 19k,67.747899,67.159664,-0.588235
Abstract First Sentence - 19k,73.218487,71.504202,-1.714286


In [54]:
oldColList = ['concat_19k_v1_old_cosSim',
'concat_19k_v2_old_cosSim']
indNames = ['Complex + Transe + Abstract FirstSent 19k', 'Text7, Text2, Complex, Transe, Abstract, Abstract FirstSent']
compareEmbeddings(wordSim353AnnotDF_New_Merged_DF, oldColList, [colMappers[col] for col in oldColList], indNames)

,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase
Complex + Transe + Abstract FirstSent 19k,66.176471,66.462185,0.285714
Combined,68.756303,69.033613,0.277311
"Text7, Text2, Complex, Transe, Abstract, Abstract FirstSent",69.630252,69.327731,-0.302521


In [56]:
list(filter(lambda p: 'old_cosSim' in p, wordSim353AnnotDF_New_Merged_DF.columns))

['text_7props_19k_old_cosSim',
 'text_2props_19k_old_cosSim',
 'complex_19k_old_cosSim',
 'transe_19k_old_cosSim',
 'abstract_19k_old_cosSim',
 'abstract_firstSent_19k_old_cosSim',
 'complex_probase_old_cosSim',
 'transe_probase_old_cosSim',
 'complex_probase_19k_old_cosSim',
 'transe_probase_19k_old_cosSim',
 'concat_19k_v1_old_cosSim',
 'concat_probase_19k_v1_old_cosSim',
 'concat_19k_v2_old_cosSim',
 'concat_probase_19k_v2_old_cosSim']

In [271]:
oldColList = ['text_7props_19k_old_cosSim',
 'text_2props_19k_old_cosSim',
 'complex_19k_old_cosSim',
 'transe_19k_old_cosSim',
 'abstract_19k_old_cosSim',
 'abstract_firstSent_19k_old_cosSim',
 'complex_probase_old_cosSim',
 'transe_probase_old_cosSim',
 'complex_probase_19k_old_cosSim',
 'transe_probase_19k_old_cosSim',
 'concat_19k_v1_old_cosSim',
 'concat_probase_19k_v1_old_cosSim',
 'concat_19k_v2_old_cosSim',
 'concat_probase_19k_v2_old_cosSim']
indNames = ['Text 7 props - 19k', 'Text 2 props - 19k', 'Complex - 19k', 'Transe - 19k', 
            'Abstract - 19k', 'Abstract First Sentence - 19k', 'Complex - Probase', 'Transe - Probase', 
            'Complex - Probase-19k', 'Transe - Probase-19k', 'Complex-Transe-AbsFirstSent - 19k',
            'Complex-Transe-AbsFirstSent - Probase-19k', '6 embeddings - 19k',
            '6 embeddings - Probase-19k']
compareEmbeddings(wordSim353AnnotDF_New_Merged_DF, oldColList, [colMappers[col] for col in oldColList], indNames)


KeyboardInterrupt



In [127]:
oldColList = ['text_7props_19k_old_cosSim',
 'text_2props_19k_old_cosSim',
 'complex_19k_old_cosSim',
 'transe_19k_old_cosSim',
 'abstract_19k_old_cosSim',
 'abstract_firstSent_19k_old_cosSim',
 'complex_probase_old_cosSim',
 'transe_probase_old_cosSim',
 'complex_probase_19k_old_cosSim',
 'transe_probase_19k_old_cosSim',
 'concat_19k_v1_old_cosSim',
 'concat_probase_19k_v1_old_cosSim',
 'concat_19k_v2_old_cosSim',
 'concat_probase_19k_v2_old_cosSim']
indNames = ['Text 7 props - 19k', 'Text 2 props - 19k', 'Complex - 19k', 'Transe - 19k', 
            'Abstract - 19k', 'Abstract First Sentence - 19k', 'Complex - Probase', 'Transe - Probase', 
            'Complex - Probase-19k', 'Transe - Probase-19k', 'Complex-Transe-AbsFirstSent - 19k',
            'Complex-Transe-AbsFirstSent - Probase-19k', '6 embeddings - 19k',
            '6 embeddings - Probase-19k']
oldColPairs = list(zip(oldColList, indNames))

In [151]:
print("\n".join(indNames))

Text 7 props - 19k
Text 2 props - 19k
Complex - 19k
Transe - 19k
Abstract - 19k
Abstract First Sentence - 19k
Complex - Probase
Transe - Probase
Complex - Probase-19k
Transe - Probase-19k
Complex-Transe-AbsFirstSent - 19k
Complex-Transe-AbsFirstSent - Probase-19k
6 embeddings - 19k
6 embeddings - Probase-19k


In [71]:
comb(len(oldColPairs), 9)

2002

In [72]:
2**(len(oldColPairs))

16384

In [ ]:
# accs = []
# for r in tqdm(range(1,len(oldColPairs)+1)):
#     for comb in tqdm(combinations(oldColPairs, r)):
#         print(len(comb))
# #         oldAcc = trainAndFindAccuracy(wordSim353AnnotDF_New_Merged_DF, [col[0] for col in comb])
# #         newAcc = trainAndFindAccuracy(wordSim353AnnotDF_New_Merged_DF, [colMappers[col[0]] for col in comb])
#         print(" & ".join([col[1] for col in comb]).count("&"))
# #         accs.append((indName, oldAcc, newAcc))
        

In [316]:
from joblib import Parallel, delayed
def generateCombAccuracies(oldColPairs, r):
    accs = []
    for comb in tqdm(combinations(oldColPairs, r)):
        assert len(comb) == r
        oldAcc = trainAndFindAccuracy(wordSim353AnnotDF_New_Merged_DF, [col[0] for col in comb])
        newAcc = trainAndFindAccuracy(wordSim353AnnotDF_New_Merged_DF, [colMappers[col[0]] for col in comb])
        indName = " & ".join([col[1] for col in comb])
        accs.append((indName, oldAcc, newAcc))
    return accs
    
results = Parallel(n_jobs=14)(delayed(generateCombAccuracies)(oldColPairs, i) for i in tqdm(range(1,len(oldColPairs)+1)))
print(results)

  0%|          | 0/14 [00:00<?, ?it/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [317]:
# from joblib import Parallel, delayed
# def generateCombAccuracies(oldColPairs, r):
#     accs = []
#     for comb in tqdm(combinations(oldColPairs, r)):
#         assert len(comb) == r
#         oldAcc = trainAndFindAccuracyRF(wordSim353AnnotDF_New_Merged_DF, [col[0] for col in comb])
#         newAcc = trainAndFindAccuracyRF(wordSim353AnnotDF_New_Merged_DF, [colMappers[col[0]] for col in comb])
#         indName = " & ".join([col[1] for col in comb])
#         accs.append((indName, oldAcc, newAcc))
#     return accs
    
# results = Parallel(n_jobs=14)(delayed(generateCombAccuracies)(oldColPairs, i) for i in tqdm(range(1,len(oldColPairs)+1)))
# print(results)

In [318]:
results_flat = [item for sublist in results for item in sublist]

In [319]:
# results_flat == results_flat1

In [320]:
allCombs = (pd.DataFrame(results_flat, columns = ['Combination','Accuracy (in %) of old embeddings compared to annotated category', 'Accuracy (in %) of new embeddings compared to annotated category']).set_index('Combination') * 100).reset_index()

In [321]:
allCombs['Increase'] = allCombs['Accuracy (in %) of new embeddings compared to annotated category'] - allCombs['Accuracy (in %) of old embeddings compared to annotated category']

In [322]:
allCombs['count'] = allCombs['Combination'].apply(lambda p: p.count("&")+1)

In [323]:
allCombs.to_csv('../data/wordsim353_all_combinations_SVM_accuracies.csv', index=None)

In [324]:
allCombs = pd.read_csv('../data/wordsim353_all_combinations_SVM_accuracies.csv')

In [325]:
allCombs.head()

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
0,Text 7 props - 19k,65.428571,65.126050,-0.302521,1
1,Text 2 props - 19k,64.554622,63.680672,-0.873950,1
2,Complex - 19k,64.579832,65.445378,0.865546,1
3,Transe - 19k,64.579832,66.890756,2.310924,1
4,Abstract - 19k,67.739496,68.050420,0.310924,1


In [326]:
print(f"Determined accuracies of {len(allCombs)} combinations of embedding scores based on the length {len(oldColPairs)}")

Determined accuracies of 16383 combinations of embedding scores based on the length 14


In [327]:
pd.options.display.max_colwidth = 0

In [328]:
allCombs.sort_values(by=['Accuracy (in %) of old embeddings compared to annotated category'], ascending=False).head(10)

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
5,Abstract First Sentence - 19k,72.705882,71.537815,-1.168067,1
30,Text 2 props - 19k & Abstract First Sentence - 19k,71.840336,70.966387,-0.873950,2
204,Text 2 props - 19k & Abstract - 19k & Abstract First Sentence - 19k,71.840336,70.378151,-1.462185,3
18,Text 7 props - 19k & Abstract First Sentence - 19k,71.840336,71.848739,0.008403,2
60,Abstract - 19k & Abstract First Sentence - 19k,71.546218,71.243697,-0.302521,2
12350,Text 2 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Transe - Probase-19k & Complex-Transe-AbsFirstSent - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k & 6 embeddings - Probase-19k,70.974790,69.504202,-1.470588,8
12329,Text 2 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex - Probase & Complex-Transe-AbsFirstSent - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k & 6 embeddings - Probase-19k,70.974790,70.075630,-0.899160,8
12154,Text 2 props - 19k & Transe - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k & 6 embeddings - Probase-19k,70.974790,70.084034,-0.890756,8
11944,Text 2 props - 19k & Complex - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k & 6 embeddings - Probase-19k,70.974790,69.781513,-1.193277,8
12349,Text 2 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex - Probase-19k & Complex-Transe-AbsFirstSent - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k & 6 embeddings - Probase-19k,70.974790,69.210084,-1.764706,8


In [329]:
allCombs.sort_values(by=['Accuracy (in %) of new embeddings compared to annotated category'], ascending=False).head(10)

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
18,Text 7 props - 19k & Abstract First Sentence - 19k,71.840336,71.848739,0.008403,2
5318,Text 2 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex - Probase & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - Probase-19k,70.394958,71.546218,1.151261,6
5,Abstract First Sentence - 19k,72.705882,71.537815,-1.168067,1
51,Transe - 19k & Abstract First Sentence - 19k,68.630252,71.521008,2.890756,2
108,Text 7 props - 19k & Text 2 props - 19k & Abstract First Sentence - 19k,69.210084,71.252101,2.042017,3
694,Text 7 props - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - 19k & 6 embeddings - 19k,70.084034,71.252101,1.168067,4
8035,Text 7 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Transe - Probase & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k & 6 embeddings - Probase-19k,70.092437,71.252101,1.159664,7
7049,Text 7 props - 19k & Text 2 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Transe - Probase & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - Probase-19k,70.386555,71.252101,0.865546,7
1916,Text 7 props - 19k & Transe - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k,69.798319,71.252101,1.453782,5
11327,Text 7 props - 19k & Transe - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex - Probase & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k & 6 embeddings - Probase-19k,70.100840,71.252101,1.151261,8


In [330]:
allCombs.sort_values(by=['Increase'], ascending=False).head(10)

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
1041,Complex - 19k & Abstract - 19k & Complex - Probase-19k & Transe - Probase-19k,66.285714,69.193277,2.907563,4
2828,Complex - 19k & Abstract - 19k & Complex - Probase & Transe - Probase & Complex - Probase-19k,65.714286,68.613445,2.899160,5
51,Transe - 19k & Abstract First Sentence - 19k,68.630252,71.521008,2.890756,2
9440,Complex - 19k & Abstract - 19k & Complex - Probase & Transe - Probase & Complex - Probase-19k & Transe - Probase-19k & 6 embeddings - Probase-19k,66.563025,69.184874,2.621849,7
2701,Complex - 19k & Transe - 19k & Abstract - 19k & Complex - Probase-19k & Transe - Probase-19k,65.714286,68.327731,2.613445,5
1035,Complex - 19k & Abstract - 19k & Transe - Probase & Complex - Probase-19k,66.285714,68.899160,2.613445,4
1030,Complex - 19k & Abstract - 19k & Complex - Probase & Transe - Probase-19k,66.285714,68.899160,2.613445,4
2689,Complex - 19k & Transe - 19k & Abstract - 19k & Complex - Probase & Complex - Probase-19k,65.714286,68.319328,2.605042,5
2834,Complex - 19k & Abstract - 19k & Complex - Probase & Complex - Probase-19k & Transe - Probase-19k,65.714286,68.319328,2.605042,5
1026,Complex - 19k & Abstract - 19k & Abstract First Sentence - 19k & 6 embeddings - 19k,68.647059,71.226891,2.579832,4


In [331]:
allCombs[allCombs['count'] == 1].sort_values(by=['Accuracy (in %) of new embeddings compared to annotated category'], ascending=False).head(10)

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
5,Abstract First Sentence - 19k,72.705882,71.537815,-1.168067,1
12,6 embeddings - 19k,70.092437,71.252101,1.159664,1
13,6 embeddings - Probase-19k,70.092437,70.672269,0.579832,1
4,Abstract - 19k,67.739496,68.050420,0.310924,1
11,Complex-Transe-AbsFirstSent - Probase-19k,67.470588,67.176471,-0.294118,1
10,Complex-Transe-AbsFirstSent - 19k,67.470588,66.899160,-0.571429,1
3,Transe - 19k,64.579832,66.890756,2.310924,1
9,Transe - Probase-19k,64.579832,66.890756,2.310924,1
7,Transe - Probase,64.579832,66.596639,2.016807,1
8,Complex - Probase-19k,64.579832,65.445378,0.865546,1


## RF

In [250]:
from joblib import Parallel, delayed
def generateCombAccuracies(oldColPairs, r):
    accs = []
    for comb in tqdm(combinations(oldColPairs, r)):
        assert len(comb) == r
        oldAcc = trainAndFindAccuracyRF(wordSim353AnnotDF_New_Merged_DF, [col[0] for col in comb])
        newAcc = trainAndFindAccuracyRF(wordSim353AnnotDF_New_Merged_DF, [colMappers[col[0]] for col in comb])
        indName = " & ".join([col[1] for col in comb])
        accs.append((indName, oldAcc, newAcc))
    return accs
    
results1 = Parallel(n_jobs=14)(delayed(generateCombAccuracies)(oldColPairs, i) for i in tqdm(range(1,len(oldColPairs)+1)))
print(results1)

  0%|          | 0/14 [00:00<?, ?it/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [263]:
results == results1

True

In [262]:
results_flat1 = [item for sublist in results1 for item in sublist]

In [252]:
allCombs1 = (pd.DataFrame(results_flat1, columns = ['Combination','Accuracy (in %) of old embeddings compared to annotated category', 'Accuracy (in %) of new embeddings compared to annotated category']).set_index('Combination') * 100).reset_index()

In [253]:
allCombs1['Increase'] = allCombs1['Accuracy (in %) of new embeddings compared to annotated category'] - allCombs['Accuracy (in %) of old embeddings compared to annotated category']

In [254]:
allCombs1['count'] = allCombs1['Combination'].apply(lambda p: p.count("&")+1)

In [255]:
allCombs1.head()

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
0,Text 7 props - 19k,65.126050,63.672269,-1.453782,1
1,Text 2 props - 19k,61.899160,63.344538,1.445378,1
2,Complex - 19k,66.613445,64.277311,-2.336134,1
3,Transe - 19k,66.613445,66.310924,-0.302521,1
4,Abstract - 19k,68.899160,66.008403,-2.890756,1


In [256]:
print(f"Determined accuracies of {len(allCombs1)} combinations of embedding scores based on the length {len(oldColPairs)}")

Determined accuracies of 16383 combinations of embedding scores based on the length 14


In [257]:
pd.options.display.max_colwidth = 0

In [258]:
allCombs1.sort_values(by=['Accuracy (in %) of old embeddings compared to annotated category'], ascending=False).head(10)

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
5,Abstract First Sentence - 19k,73.285714,70.075630,-3.210084,1
60,Abstract - 19k & Abstract First Sentence - 19k,72.117647,68.327731,-3.789916,2
138,Text 7 props - 19k & Abstract - 19k & Abstract First Sentence - 19k,72.100840,70.361345,-1.739496,3
151,Text 7 props - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - 19k,71.831933,70.378151,-1.453782,3
152,Text 7 props - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k,71.831933,71.252101,-0.579832,3
2114,Text 7 props - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k & 6 embeddings - Probase-19k,71.815126,69.789916,-2.025210,5
2001,Text 7 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - Probase-19k,71.815126,69.798319,-2.016807,5
2000,Text 7 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k,71.815126,70.378151,-1.436975,5
1999,Text 7 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - 19k & 6 embeddings - Probase-19k,71.815126,69.495798,-2.319328,5
1998,Text 7 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - 19k & 6 embeddings - 19k,71.815126,69.487395,-2.327731,5


In [259]:
allCombs1.sort_values(by=['Accuracy (in %) of new embeddings compared to annotated category'], ascending=False).head(10)

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
681,Text 7 props - 19k & Abstract First Sentence - 19k & Transe - Probase & Complex-Transe-AbsFirstSent - Probase-19k,70.663866,71.260504,0.596639,4
218,Text 2 props - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k,71.226891,71.252101,0.025210,3
1575,Text 7 props - 19k & Text 2 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k,71.235294,71.252101,0.016807,5
152,Text 7 props - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k,71.831933,71.252101,-0.579832,3
3884,Text 7 props - 19k & Text 2 props - 19k & Abstract First Sentence - 19k & Complex - Probase-19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k,68.310924,71.235294,2.924370,6
129,Text 7 props - 19k & Transe - 19k & Abstract First Sentence - 19k,69.764706,71.235294,1.470588,3
217,Text 2 props - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - 19k,71.226891,70.974790,-0.252101,3
604,Text 7 props - 19k & Transe - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k,71.529412,70.966387,-0.563025,4
7207,Text 7 props - 19k & Text 2 props - 19k & Abstract First Sentence - 19k & Transe - Probase-19k & Complex-Transe-AbsFirstSent - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - Probase-19k,68.605042,70.966387,2.361345,7
690,Text 7 props - 19k & Abstract First Sentence - 19k & Transe - Probase-19k & Complex-Transe-AbsFirstSent - Probase-19k,70.663866,70.966387,0.302521,4


In [260]:
allCombs1.sort_values(by=['Increase'], ascending=False).head(10)

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
13048,Text 7 props - 19k & Text 2 props - 19k & Complex - 19k & Transe - 19k & Abstract First Sentence - 19k & Complex - Probase & Transe - Probase & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k,64.252101,70.352941,6.100840,9
13084,Text 7 props - 19k & Text 2 props - 19k & Complex - 19k & Transe - 19k & Abstract First Sentence - 19k & Transe - Probase & Transe - Probase-19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k,64.252101,69.210084,4.957983,9
14022,Text 7 props - 19k & Complex - 19k & Abstract First Sentence - 19k & Transe - Probase & Transe - Probase-19k & Complex-Transe-AbsFirstSent - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k & 6 embeddings - Probase-19k,64.529412,69.176471,4.647059,9
13861,Text 7 props - 19k & Complex - 19k & Transe - 19k & Abstract First Sentence - 19k & Complex - Probase & Transe - Probase-19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k & 6 embeddings - Probase-19k,64.243697,68.890756,4.647059,9
7551,Text 7 props - 19k & Complex - 19k & Abstract - 19k & Complex - Probase & Complex - Probase-19k & Transe - Probase-19k & 6 embeddings - Probase-19k,63.092437,67.722689,4.630252,7
13058,Text 7 props - 19k & Text 2 props - 19k & Complex - 19k & Transe - 19k & Abstract First Sentence - 19k & Complex - Probase & Complex - Probase-19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k,64.252101,68.605042,4.352941,9
6500,Text 7 props - 19k & Text 2 props - 19k & Complex - 19k & Transe - 19k & Abstract - 19k & Complex - Probase-19k & 6 embeddings - Probase-19k,63.689076,68.033613,4.344538,7
7537,Text 7 props - 19k & Complex - 19k & Abstract - 19k & Complex - Probase & Transe - Probase & Complex - Probase-19k & 6 embeddings - Probase-19k,63.092437,67.436975,4.344538,7
7570,Text 7 props - 19k & Complex - 19k & Abstract - 19k & Transe - Probase & Complex - Probase-19k & Transe - Probase-19k & 6 embeddings - 19k,63.092437,67.428571,4.336134,7
6499,Text 7 props - 19k & Text 2 props - 19k & Complex - 19k & Transe - 19k & Abstract - 19k & Complex - Probase-19k & 6 embeddings - 19k,63.689076,68.025210,4.336134,7


In [261]:
allCombs1[allCombs1['count'] == 1].sort_values(by=['Accuracy (in %) of new embeddings compared to annotated category'], ascending=False).head(10)

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
11,Complex-Transe-AbsFirstSent - Probase-19k,70.042017,70.630252,0.588235,1
5,Abstract First Sentence - 19k,73.285714,70.075630,-3.210084,1
10,Complex-Transe-AbsFirstSent - 19k,70.042017,68.302521,-1.739496,1
13,6 embeddings - Probase-19k,68.907563,68.042017,-0.865546,1
12,6 embeddings - 19k,68.907563,66.857143,-2.050420,1
6,Complex - Probase,66.613445,66.596639,-0.016807,1
7,Transe - Probase,66.613445,66.596639,-0.016807,1
3,Transe - 19k,66.613445,66.310924,-0.302521,1
9,Transe - Probase-19k,66.613445,66.008403,-0.605042,1
4,Abstract - 19k,68.899160,66.008403,-2.890756,1


# Generate Embeddings Files

In [133]:
wordList = pd.concat([wordSim353AnnotDF_New2[['word1_kg_id', 'Word 1']], wordSim353AnnotDF_New2[['word2_kg_id', 'Word 2']].rename(columns={'Word 2': 'Word 1', 'word2_kg_id': 'word1_kg_id'})])

In [134]:
wordList = wordList[~wordList.duplicated()]

In [135]:
wordList['word1_kg_id'].apply(lambda p: "\t".join([str(p1) for p1 in embedDict[p].tolist()])).to_csv('../data/wordsim353_embeddings.tsv', index=False, header=None)

In [136]:
wordList['word1_kg_id'].apply(lambda p: "\t".join([str(p1) for p1 in newEmbedDict[p].tolist()])).to_csv('../data/wordsim353_embeddings_new_attempt1.tsv', index=False, header=None)

In [137]:
wordList['Word 1'].to_csv('../data/wordsim353_embeddings_words.tsv', index=False, header=None)